## 修改记录

小学期时间点：
- 6 月29 日完成实习导师、题目与学生三方匹配；
- 7 月13 日-25 日，开展实习内容；
- 8 月31 日之前提交实习报告。

#### 截止7.19 
- 学习CNN的基础知识，主要参考了Deep Learning with Python(Author: Francois Chollet)，深度学习入门（作者：斋藤康毅）和lamda实验室魏秀参的解析卷积神经网络。同时也在网络上参考了一些相关的官方文档（尤其是pytorch的示例）和博客。
- 利用预训练模型实现迁移学习
- 问题：
```
  1. 训练集准确率小于测试集
  2. 没有实现随机划分
```
- 解决方案：
```
  1. 交叉验证，看下是否还是训练精度低于测试精度；
  2. 随机选择数据作为测试集；
  3. 去掉0-30度的数据看下实验效果；
  4. 查找一下是否有解决dropout局限性的方法；
  5. 整理算法的框架图及伪代码
```


#### 7.20
- 尝试实现训练集和测试集的随机划分，以及K折交叉验证
- 问题：
```
  1. 一开始采用pytoch的包完成数据读取，预处理和分块；
  2. 在利用ImageFolder生成dataset后 没有现有函数对其进行transform（因为训练集和测试集采用
  不同的方法transform）
  3. pytorch没有现成API实现交叉验证
```
- 解决方案：
```
  1. 考虑采用sklearn完成数据的处理，但还没找到合适的transform 方法
  2. 随机划分可以使用pytorch的  
  torch.utils.data.random_split 或者  
  sklearn.model_selection.train_test_split
  3. 使用 KFold 进行K折划分
  4. 重写训练函数，K折交叉验证参考：  
  https://blog.csdn.net/foneone/article/details/104445320  ；  
  https://blog.csdn.net/Pl_Sun/article/details/106975414  
```

#### 7.21

- 添加随机划分和K折交叉验证（都已完成）
- 关于交叉验证的一些思考 https://www.jianshu.com/p/f14826061612
- 还没测试去掉30度以下数据的效果
- 问题：
```
  1. 标准化参数的选择，transforms.Normalize，目前灰度图采用的是Mnist的参数
  2. batch_size的选择，一般可能偏大时效果变好（不一定），此数据集较小，目前认为 4 比较好
  3. 依然存在val_acc > train_acc (根据目前训练过程，猜测和数据集划分方式有关，随机划分时
  验证集每一类都是平均的，K折交叉验证时出现概率更小。数据集较小，容易出现巧合)
  4. 以下是关于准确率的（7.21使用的都是night_data，7.20使用的是morning_data）  
  K折（10折）交叉验证很多个分组的效果明显较差（大多在40%以下）
  未训练完，未测试morning data
  对于随机划分数据集的准确率在 9:1划分是55%，8:2划分是76%
  利用night_data训练的模型去测试所有morning data，准确率很低（百分之几）
  morning_data和night_data存在较大偏差？
```




#### 7.22
- ~~打牌（误）~~
- 跑交叉验证的模型，因为一些小错误导致最后的结果都没能保存下来
- 问题：  
```  
  1. 交叉验证综合起来看准确率下降明显（目前看大概30%）， 5折和10折划分跑出来结果相差较大
  2. 今天采用的batch_size是16，感觉没有明显的差别
  3. 交叉验证中大多情况下，其中会有一两折的的准确率非常高，大概在80%-90%。
```

#### 7.23
- 添加了分层的交叉验证划分方式，目前来看效果改善很明显
- 参数如下，分层随机抽取，10折，batch_size=4
```
  1. 晚上数据训练最佳acc=0.8947，平均acc=0.236
  2. 上午数据训练最佳acc=1, 平均acc=0.957
  3. 利用晚上的最佳模型测试上午数据，对了14个图，acc=0.073
  4. 利用上午的最佳模型测试晚上数据，对了20个图，acc=0.104
  5. 可以初步推测上午和晚上的数据在该模型中存在较大的差别
```

#### 7.24 - 7.25
- 针对之前遇到的问题进行了一定分析
- 将所有代码重新梳理了一遍，检查是否存在不明显的逻辑错误，同时与官方文档（pytorch）进行比对
- 针对图像进行了一定增强处理，使得黑色背景（无信息部分）更少
- 测试了Adagrad的效果，因为失误清除了测试结果没有记录，效果与SGD差不多
```
  1. 改进不明显
  2. 可能选用的DenseNet不太适合？
```


#### 7.28-7.29
- 在开源数据集中，选择了MNIST和CIFAR10
- 在已有代码的基础上，重新编写了网络搭建和模型训练部分 其他进行微调
- MNIST是灰度图，数据集较为简单。CIFAR-10是RGB图，数据集更偏向人眼图片 更为复杂。
- DenseNet测试以上两个数据集和论文中所述基本一致。使用自己的网络结构测试时，结果也在预期内。


#### 完成小学期报告
- 主要分成四个部分
```
  1. CNN基础知识的学习，以及在代码编写中遇到的一些问题和解决
  2. 所采用的网络结构（DenseNet以及两个简单的CNN网络）及介绍
  3. 模型的训练和结果
  4. 学习日记和总结
```

## 依赖包
<!-- **Author**: `zwy` -->


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from sklearn.model_selection import KFold, StratifiedShuffleSplit

import random
import shutil

plt.ion()   # 交互模式可以动态显示图像

### 硬件选择 GPU

In [ ]:
# GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

## 数据加载及处理


### 下载数据集
可以从Google Drive / Github / 其他数据地址下载数据

In [ ]:
!rm -rf CNN
!git clone https://github.com/Zwysun/CNN.git
!ls

Cloning into 'CNN'...
remote: Enumerating objects: 693, done.
remote: Counting objects: 100% (693/693), done.
remote: Compressing objects: 100% (551/551), done.
remote: Total 693 (delta 160), reused 670 (delta 140), pack-reused 0
Receiving objects: 100% (693/693), 31.28 MiB | 15.92 MiB/s, done.
Resolving deltas: 100% (160/160), done.
CNN  sample_data


In [ ]:
!rm -rf CNN/dataset/morning/val
!rm -rf CNN/dataset/morning/train

### 数据预处理
利用transforms对图片进行预处理，可以分别针对训练集和验证集采取不同的处理方法


#### 比例随机划分 数据集和验证集

In [ ]:
data_dir = 'CNN/dataset/night'    # 数据目录
ratio_train = 0.7                 # 训练集比例
 
def data_random_split(current_dir,ratio_train):
    '''
    将当前文件夹中文件按一定比例分成train和test连个列表，列表存放文件名
    '''
    data_listdir = os.listdir(current_dir)
    random.shuffle(data_listdir)
    train_len = int(len(data_listdir)*ratio_train)
    train_listdir = data_listdir[:train_len]
    train_listdir = data_listdir[:train_len]
    test_listdir  = data_listdir[train_len:]
    return train_listdir,test_listdir
 
def data_generator(root,data_dir,ratio_train=0.8):
    '''
    root：输入你的data目录
    datadir：你要创建好的的文件夹，将会生成train、和val
    ratio_train:train_data占总数据的比例
    '''
    listdir = os.listdir(root)
    train_data_dir = os.path.join(data_dir, "train")
    test_data_dir = os.path.join(data_dir, "val")
    os.makedirs(train_data_dir)
    os.makedirs(test_data_dir)
    for name in listdir:
        print(name)
        current_dir = os.path.join(root, name)
        print(current_dir)
        train_dir_a,test_dir_a = data_random_split(current_dir,ratio_train)
        train_listdir_c = os.path.join(train_data_dir,name)
        test_listdir_c  = os.path.join(test_data_dir,name)
        a=1 if os.path.exists(train_listdir_c) else os.makedirs(train_listdir_c)
        a=1 if os.path.exists(test_listdir_c) else os.makedirs(test_listdir_c)
        for img in train_dir_a:
            train_listdir_b = os.path.join(current_dir, img) 
            train_listdir_d = os.path.join(train_listdir_c, img)
            train_dir_b = shutil.copy(train_listdir_b,train_listdir_d)
        for img in test_dir_a:
            test_listdir_b  = os.path.join(current_dir, img)
            test_listdir_d  = os.path.join(test_listdir_c, img)
            test_dir_b = shutil.copy(test_listdir_b,test_listdir_d)
                
    print('ok')

# 随机划分为训练集和验证集
data_generator(data_dir, data_dir, ratio_train)
 

In [ ]:
# !cd train_data
# !ls -lR |grep -v ^d|awk '{print $9}'

In [ ]:
# 训练集 扩充及正则化
# 验证集 仅正则化
data_transforms = {
    'train': transforms.Compose([
        transforms.Grayscale(1),      # 转换为灰度图
        transforms.CenterCrop(1080),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),    # 增广
        transforms.ToTensor(),
        # transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])    # RGB标准化参数(Imagenet Dateset)
        transforms.Normalize([0.1307], [0.3081])      # 灰度图标准化参数(Mnist)
    ]),
    'val': transforms.Compose([
        transforms.Grayscale(1),
        transforms.CenterCrop(1080),
        transforms.Resize(224),
        transforms.ToTensor(),
        # transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        transforms.Normalize([0.1307], [0.3081])      # 灰度图标准化参数
    ]),
}

# full_datasets = datasets.ImageFolder(data_dir)
# train_size = int(0.8 * len(full_datasets))   # 训练集划分比例  还有点问题
# test_size = len(full_datasets) - train_size
# train_dataset, test_dataset = torch.utils.data.random_split(full_datasets, [train_size, test_size])

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes
print (dataset_sizes)
print (image_datasets)
print (class_names)

{'train': 133, 'val': 57}
{'train': Dataset ImageFolder
    Number of datapoints: 133
    Root location: CNN/dataset/night/train
    StandardTransform
Transform: Compose(
               Grayscale(num_output_channels=1)
               CenterCrop(size=(1080, 1080))
               RandomResizedCrop(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=PIL.Image.BILINEAR)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
               Normalize(mean=[0.1307], std=[0.3081])
           ), 'val': Dataset ImageFolder
    Number of datapoints: 57
    Root location: CNN/dataset/night/val
    StandardTransform
Transform: Compose(
               Grayscale(num_output_channels=1)
               CenterCrop(size=(1080, 1080))
               Resize(size=224, interpolation=PIL.Image.BILINEAR)
               ToTensor()
               Normalize(mean=[0.1307], std=[0.3081])
           )}
['0', '10', '15', '20', '25', '30', '35', '40', '45', '5', '50', '55', '60', '65',

#### K折交叉验证(的数据处理)

In [ ]:
data_dir = 'CNN/dataset/morning'    # 数据目录

data_transforms = transforms.Compose([
        transforms.Grayscale(1),      # 转换为灰度图
        transforms.CenterCrop(1080),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),    # 增广
        transforms.ToTensor(),
        # transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])    # RGB标准化参数(Imagenet Dateset)
        transforms.Normalize([0.1307], [0.3081])      # 灰度图标准化参数(Mnist)
    ])

image_datasets = datasets.ImageFolder(data_dir, data_transforms)

class_names = image_datasets.classes
dataset_sizes = len(image_datasets)
print ('Total:', dataset_sizes)
print (class_names)

Total: 190
['35', '40', '45', '5', '50', '55', '60', '65', '70', '75', '80', '85', '90', 'low']


In [ ]:
# 不要运行，测试划分
X=np.arange(380).reshape(190,2)

for train_index,test_index in kf.split(X):
    print('train_index %s, test_index %s'%(train_index, test_index))

train_sampler = torch.utils.data.SubsetRandomSampler(train_index)
valid_sampler = torch.utils.data.SubsetRandomSampler(test_index)

### 显示部分图片


In [ ]:
# 交叉验证这 用不了
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    # mean = np.array([0.485, 0.456, 0.406])
    # std = np.array([0.229, 0.224, 0.225])   # RGB
    mean = np.array([0.1307])
    std = np.array([0.3081])   # 灰度图
    
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

# Get a batch of training data
inputs, classes = next(iter(dataloaders))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

imshow(out, title=[class_names[x] for x in classes])

## 模型训练

In [ ]:
def train_model(model, dataloaders, criterion, optimizer, scheduler, datasize, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / datasize[phase]
            epoch_acc = running_corrects.double() / datasize[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'train':
              scheduler.step()

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, best_acc

In [ ]:
# K折交叉验证

def k_fold_train(k, dataset, model, batch_size, criterion, optimizer, scheduler, num_epochs=25):
    # kf = KFold(k, shuffle=True)

    # 分层划分，避免出现某一层全在训练集/验证集的情况
    kf = StratifiedShuffleSplit(n_splits=k,train_size=1-1/k,test_size=1/k,random_state=True)
    
    i = 1
    valid_acc_sum = 0    
    Kfold_best_acc = 0.0
    kfold_model_wts = copy.deepcopy(model.state_dict())
    best_model_dict = kfold_model_wts

    X=np.arange(len(dataset)*2).reshape(len(dataset),2)
    y=np.arange(190)
    for j in range(19):
      y[j*10:j*10+10]=j

    for train_index,val_index in kf.split(X, y):
      print('*'*25,'第',i,'折','*'*25)
      datasize = {'train':len(train_index), 'val':len(val_index)}
      print('train:',datasize['train'],'val:',datasize['val'])
      print('train_index %s, test_index %s'%(train_index, val_index))

      i += 1

      train_sampler = torch.utils.data.SubsetRandomSampler(train_index)
      valid_sampler = torch.utils.data.SubsetRandomSampler(val_index)
      dataloaders = {'train': torch.utils.data.DataLoader(image_datasets, 
                  batch_size=batch_size, num_workers=4, sampler=train_sampler),
              'val': torch.utils.data.DataLoader(image_datasets,
                  batch_size=batch_size, num_workers=4, sampler=valid_sampler)
              }
      model.load_state_dict(kfold_model_wts) #每一折训练前 都重置为初始状态

      model, k_acc = train_model(model, dataloaders, criterion, optimizer, scheduler, datasize, num_epochs)
      valid_acc_sum += float(k_acc)
      if Kfold_best_acc < k_acc:
        Kfold_best_acc = k_acc
        best_model_dict = copy.deepcopy(model.state_dict()) # 保存最佳的训练参数
    
    valid_acc = valid_acc_sum/(i-1)
    print('*'*25,'END','*'*25)
    print('the best acc in kfold', float(Kfold_best_acc))
    print('teh average val acc', valid_acc)
    model.load_state_dict(best_model_dict)
    return model

## 搭建网络结构

In [ ]:
model_ft = models.densenet169(pretrained=True)
num_ftrs = model_ft.classifier.in_features
model_ft.classifier = nn.Linear(num_ftrs, 19, bias=True)  # 输出参数修改

num_ochannels = model_ft.features.conv0.out_channels
model_ft.features.conv0 = nn.Conv2d(1, num_ochannels, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False) # 输入参数修改

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)
# print (model_ft)

Downloading: "https://download.pytorch.org/models/densenet169-b2777c0a.pth" to /root/.cache/torch/hub/checkpoints/densenet169-b2777c0a.pth


NameError: ignored

In [ ]:
# 搭配下面工具，轻松实现可视化
import torch.onnx
from torch.autograd import Variable

x = Variable(torch.randn(1, 1, 224, 224)).cuda()
torch_out = torch.onnx.export(model_ft, x, "test.onnx", export_params=True, verbose=True)

In [ ]:
# 可视化工具
https://lutzroeder.github.io/netron/

## 训练

K折交叉验证

In [ ]:
model_ft = k_fold_train(5, image_datasets, model_ft, 4, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=100)
# os.chdir("/content/drive/My Drive")
!ls
torch.save(model_ft,'light_10fold_except.pth')
!ls
# 针对最优模型再次训练, 可选
# model_ft, _ = train_model(model_ft, dataloaders, 8, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=100)

************************* 第 1 折 *************************
train: 152 val: 38
train_index [135 176  66 189 164  41   9  78  30  94 179 154  10 152 131 146  99 126
  69 132 159  18  84  25 110 115  42 163 155  74 106  96 169 156 165  91
   4 177 111  27   0  73  71  15 102  28  85   2  75 138 178  62 140  76
 187   1  14  13 139 124  21 130  80 157  45 188 121 120  81  23 118 167
  54 105 153   3 123  43 125  88 117 133  47  70 160 150   7 101  33  29
  12 113  35  11  59  20 182 148  86  57  98  68 183 149 136 145 122  19
  95 119 143  38 104  65  36  46  63  67 109 103   6  44 112 185  40  97
  77  31 168  51  82 181 186 107  58  55 144  93  52  26 129  83  61 173
  32 172 161  50 142 170 174  34], test_index [ 72 128  53  49 141 180 171 175  22  48  56 151  87 100  24  64 184  89
  79  90 116 147  60  37  17 162  16 114   8  39 108 134 166 158  92   5
 137 127]
Epoch 0/99
----------
train Loss: 2.5090 Acc: 0.2829
val Loss: 2.6938 Acc: 0.3158

Epoch 1/99
----------
train Loss: 2.3481 A

### 测试和保存

In [ ]:
from google.colab import drive
drive.mount('/content/drive')   # 挂载云盘

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
os.chdir("/content")
torch.save(model_ft,'light.pth')
!ls

CNN  light_10fold.pth  light.pth  sample_data  test.onnx


In [ ]:
os.chdir("/content/drive/My Drive")
!ls
torch.save(model_ft,'light.pth')

'2020 年日历.gsheet'  'Colab Notebooks'  'Science Journal'


In [ ]:
# 训练集 扩充及正则化
# 验证集 仅正则化
data_dir = 'CNN/dataset/night'    # 数据目录
# !rm -rf CNN/dataset/morning/train
# !rm -rf CNN/dataset/morning/val
data_transforms = transforms.Compose([
        transforms.Grayscale(1),
        transforms.CenterCrop(1080),
        transforms.Resize(224),
        transforms.ToTensor(),
        # transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        transforms.Normalize([0.1307], [0.3081])      # 灰度图标准化参数
    ])

test_datasets = datasets.ImageFolder(data_dir, data_transforms)
test_dataloaders = torch.utils.data.DataLoader(test_datasets, batch_size=4, shuffle=True, num_workers=4)

dataset_sizes = len(test_datasets)
class_names = test_datasets.classes
print (dataset_sizes)
print (test_datasets)
print (class_names)

190
Dataset ImageFolder
    Number of datapoints: 190
    Root location: CNN/dataset/night
    StandardTransform
Transform: Compose(
               Grayscale(num_output_channels=1)
               CenterCrop(size=(1080, 1080))
               Resize(size=224, interpolation=PIL.Image.BILINEAR)
               ToTensor()
               Normalize(mean=[0.1307], std=[0.3081])
           )
['0', '10', '15', '20', '25', '30', '35', '40', '45', '5', '50', '55', '60', '65', '70', '75', '80', '85', '90']


In [ ]:
test_dataloaders=dataloaders['val']

In [ ]:
#show acc
model = torch.load('light.pth')
eval_loss = 0.
eval_acc = 0.
s= 0.
with torch.no_grad():
  for i, (inputs, labels) in enumerate(test_dataloaders):
    inputs = inputs.to(device)
    labels = labels.to(device)

    outputs = model(inputs)
    _, preds = torch.max(outputs, 1)
    for j in range(inputs.size()[0]):
      #s =s + int(class_names[preds[j]])
      #print(class_names[preds[j]])
      #if int(class_names[preds[j]]) == int(labels[j]):
      if class_names[preds[j]] == class_names[int(labels[j])]:
        s = s+1
print (s)
print (s/(len(test_dataloaders) * 4))

20.0
0.10416666666666667


# output

#### g
************************* 第 1 折 *************************
train: 180 val: 10
Epoch 1/100
----------
/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
流式输出内容被截断，只能显示最后 5000 行内容。

Epoch 9/100
----------
train Loss: 2.9858 Acc: 0.0543
val Loss: 2.2544 Acc: 0.0000

Epoch 10/100
----------
train Loss: 2.9556 Acc: 0.0435
val Loss: 2.2802 Acc: 0.0833

Epoch 11/100
----------
train Loss: 2.9511 Acc: 0.0598
val Loss: 2.2826 Acc: 0.0833

Epoch 12/100
----------
train Loss: 2.9826 Acc: 0.0489
val Loss: 2.2805 Acc: 0.0000

Epoch 13/100
----------
train Loss: 2.9718 Acc: 0.0380
val Loss: 2.2297 Acc: 0.0000

Epoch 14/100
----------
train Loss: 2.9678 Acc: 0.0326
val Loss: 2.2688 Acc: 0.0000

Epoch 15/100
----------
train Loss: 2.9505 Acc: 0.0435
val Loss: 2.2342 Acc: 0.0833

Epoch 16/100
----------
train Loss: 2.9635 Acc: 0.0380
val Loss: 2.2715 Acc: 0.0000

Epoch 17/100
----------
train Loss: 2.9666 Acc: 0.0272
val Loss: 2.3357 Acc: 0.0833

Epoch 18/100
----------
train Loss: 2.9397 Acc: 0.0543
val Loss: 2.2328 Acc: 0.0833

Epoch 19/100
----------
train Loss: 2.9628 Acc: 0.0272
val Loss: 2.3843 Acc: 0.0000

Epoch 20/100
----------
train Loss: 2.9415 Acc: 0.0543
val Loss: 2.1904 Acc: 0.1667

Epoch 21/100
----------
train Loss: 2.9534 Acc: 0.0543
val Loss: 2.2940 Acc: 0.0833

Epoch 22/100
----------
train Loss: 2.9796 Acc: 0.0435
val Loss: 2.2733 Acc: 0.0000

Epoch 23/100
----------
train Loss: 2.9516 Acc: 0.0489
val Loss: 2.3172 Acc: 0.0833

Epoch 24/100
----------
train Loss: 2.9849 Acc: 0.0489
val Loss: 2.3891 Acc: 0.0000

Epoch 25/100
----------
train Loss: 2.9810 Acc: 0.0489
val Loss: 2.2641 Acc: 0.0833

Epoch 26/100
----------
train Loss: 2.9713 Acc: 0.0326
val Loss: 2.2742 Acc: 0.0000

Epoch 27/100
----------
train Loss: 2.9599 Acc: 0.0598
val Loss: 2.2678 Acc: 0.0000

Epoch 28/100
----------
train Loss: 2.9528 Acc: 0.0815
val Loss: 2.2669 Acc: 0.0000

Epoch 29/100
----------
train Loss: 2.9580 Acc: 0.0870
val Loss: 2.2459 Acc: 0.0000

Epoch 30/100
----------
train Loss: 2.9579 Acc: 0.0543
val Loss: 2.3851 Acc: 0.0000

Epoch 31/100
----------
train Loss: 2.9661 Acc: 0.0380
val Loss: 2.2857 Acc: 0.1667

Epoch 32/100
----------
train Loss: 2.9647 Acc: 0.0380
val Loss: 2.3214 Acc: 0.0000

Epoch 33/100
----------
train Loss: 2.9669 Acc: 0.0326
val Loss: 2.2380 Acc: 0.0833

Epoch 34/100
----------
train Loss: 2.9362 Acc: 0.0707
val Loss: 2.2052 Acc: 0.0000

Epoch 35/100
----------
train Loss: 2.9702 Acc: 0.0326
val Loss: 2.2526 Acc: 0.0000

Epoch 36/100
----------
train Loss: 2.9499 Acc: 0.0326
val Loss: 2.3234 Acc: 0.0000

Epoch 37/100
----------
train Loss: 2.9521 Acc: 0.0761
val Loss: 2.2595 Acc: 0.0000

Epoch 38/100
----------
train Loss: 2.9754 Acc: 0.0489
val Loss: 2.3304 Acc: 0.0000

Epoch 39/100
----------
train Loss: 2.9442 Acc: 0.0652
val Loss: 2.3096 Acc: 0.0000

Epoch 40/100
----------
train Loss: 2.9438 Acc: 0.0707
val Loss: 2.2882 Acc: 0.0000

Epoch 41/100
----------
train Loss: 2.9660 Acc: 0.0652
val Loss: 2.3883 Acc: 0.0000

Epoch 42/100
----------
train Loss: 2.9581 Acc: 0.0652
val Loss: 2.3271 Acc: 0.0000

Epoch 43/100
----------
train Loss: 2.9543 Acc: 0.0652
val Loss: 2.2966 Acc: 0.0000

Epoch 44/100
----------
train Loss: 2.9618 Acc: 0.0543
val Loss: 2.3564 Acc: 0.0000

Epoch 45/100
----------
train Loss: 2.9520 Acc: 0.0489
val Loss: 2.2223 Acc: 0.0000

Epoch 46/100
----------
train Loss: 2.9852 Acc: 0.0163
val Loss: 2.2871 Acc: 0.0000

Epoch 47/100
----------
train Loss: 2.9504 Acc: 0.0489
val Loss: 2.2706 Acc: 0.0000

Epoch 48/100
----------
train Loss: 2.9640 Acc: 0.0435
val Loss: 2.1896 Acc: 0.0000

Epoch 49/100
----------
train Loss: 2.9532 Acc: 0.0761
val Loss: 2.3182 Acc: 0.0000

Epoch 50/100
----------
train Loss: 2.9536 Acc: 0.0489
val Loss: 2.3011 Acc: 0.0000

Epoch 51/100
----------
train Loss: 2.9511 Acc: 0.0489
val Loss: 2.2948 Acc: 0.0000

Epoch 52/100
----------
train Loss: 2.9757 Acc: 0.0380
val Loss: 2.2074 Acc: 0.0833

Epoch 53/100
----------
train Loss: 2.9873 Acc: 0.0707
val Loss: 2.2511 Acc: 0.0833

Epoch 54/100
----------
train Loss: 2.9651 Acc: 0.0489
val Loss: 2.2357 Acc: 0.0833

Epoch 55/100
----------
train Loss: 2.9875 Acc: 0.0652
val Loss: 2.2204 Acc: 0.0000

Epoch 56/100
----------
train Loss: 2.9591 Acc: 0.0598
val Loss: 2.2959 Acc: 0.0833

Epoch 57/100
----------
train Loss: 2.9808 Acc: 0.0380
val Loss: 2.2486 Acc: 0.0000

Epoch 58/100
----------
train Loss: 2.9846 Acc: 0.0489
val Loss: 2.2777 Acc: 0.0000

Epoch 59/100
----------
train Loss: 2.9717 Acc: 0.0652
val Loss: 2.2913 Acc: 0.0000

Epoch 60/100
----------
train Loss: 2.9905 Acc: 0.0598
val Loss: 2.2455 Acc: 0.0000

Epoch 61/100
----------
train Loss: 2.9758 Acc: 0.0380
val Loss: 2.2013 Acc: 0.1667

Epoch 62/100
----------
train Loss: 2.9694 Acc: 0.0326
val Loss: 2.2452 Acc: 0.0833

Epoch 63/100
----------
train Loss: 2.9507 Acc: 0.0489
val Loss: 2.2457 Acc: 0.1667

Epoch 64/100
----------
train Loss: 2.9628 Acc: 0.0489
val Loss: 2.3508 Acc: 0.0000

Epoch 65/100
----------
train Loss: 2.9652 Acc: 0.0435
val Loss: 2.3018 Acc: 0.0000

Epoch 66/100
----------
train Loss: 2.9766 Acc: 0.0380
val Loss: 2.2671 Acc: 0.0000

Epoch 67/100
----------
train Loss: 2.9660 Acc: 0.0489
val Loss: 2.2801 Acc: 0.0000

Epoch 68/100
----------
train Loss: 2.9763 Acc: 0.0435
val Loss: 2.2078 Acc: 0.0000

Epoch 69/100
----------
train Loss: 2.9785 Acc: 0.0598
val Loss: 2.2309 Acc: 0.0833

Epoch 70/100
----------
train Loss: 2.9747 Acc: 0.0489
val Loss: 2.3788 Acc: 0.0000

Epoch 71/100
----------
train Loss: 2.9711 Acc: 0.0598
val Loss: 2.2041 Acc: 0.0000

Epoch 72/100
----------
train Loss: 2.9679 Acc: 0.0543
val Loss: 2.1373 Acc: 0.1667

Epoch 73/100
----------
train Loss: 2.9712 Acc: 0.0652
val Loss: 2.2989 Acc: 0.0000

Epoch 74/100
----------
train Loss: 2.9551 Acc: 0.0707
val Loss: 2.2650 Acc: 0.0000

Epoch 75/100
----------
train Loss: 2.9753 Acc: 0.0380
val Loss: 2.2441 Acc: 0.0000

Epoch 76/100
----------
train Loss: 2.9422 Acc: 0.0598
val Loss: 2.3157 Acc: 0.0833

Epoch 77/100
----------
train Loss: 2.9757 Acc: 0.0435
val Loss: 2.2965 Acc: 0.0000

Epoch 78/100
----------
train Loss: 2.9545 Acc: 0.0489
val Loss: 2.2316 Acc: 0.0000

Epoch 79/100
----------
train Loss: 2.9813 Acc: 0.0435
val Loss: 2.2218 Acc: 0.0000

Epoch 80/100
----------
train Loss: 2.9754 Acc: 0.0435
val Loss: 2.3407 Acc: 0.0833

Epoch 81/100
----------
train Loss: 2.9722 Acc: 0.0489
val Loss: 2.2668 Acc: 0.0000

Epoch 82/100
----------
train Loss: 2.9904 Acc: 0.0598
val Loss: 2.3140 Acc: 0.0000

Epoch 83/100
----------
train Loss: 2.9653 Acc: 0.0543
val Loss: 2.2582 Acc: 0.0833

Epoch 84/100
----------
train Loss: 2.9801 Acc: 0.0489
val Loss: 2.3154 Acc: 0.0000

Epoch 85/100
----------
train Loss: 2.9600 Acc: 0.0543
val Loss: 2.3552 Acc: 0.0000

Epoch 86/100
----------
train Loss: 2.9691 Acc: 0.0489
val Loss: 2.2988 Acc: 0.0833

Epoch 87/100
----------
train Loss: 2.9634 Acc: 0.0598
val Loss: 2.3138 Acc: 0.0000

Epoch 88/100
----------
train Loss: 2.9606 Acc: 0.0489
val Loss: 2.2464 Acc: 0.0000

Epoch 89/100
----------
train Loss: 2.9663 Acc: 0.0380
val Loss: 2.2564 Acc: 0.0000

Epoch 90/100
----------
train Loss: 2.9620 Acc: 0.0435
val Loss: 2.2944 Acc: 0.0000

Epoch 91/100
----------
train Loss: 2.9584 Acc: 0.0435
val Loss: 2.2866 Acc: 0.0000

Epoch 92/100
----------
train Loss: 2.9508 Acc: 0.0326
val Loss: 2.2191 Acc: 0.0833

Epoch 93/100
----------
train Loss: 2.9803 Acc: 0.0217
val Loss: 2.3061 Acc: 0.0833

Epoch 94/100
----------
train Loss: 2.9578 Acc: 0.0870
val Loss: 2.2622 Acc: 0.0000

Epoch 95/100
----------
train Loss: 2.9414 Acc: 0.0435
val Loss: 2.2914 Acc: 0.0000

Epoch 96/100
----------
train Loss: 2.9504 Acc: 0.0380
val Loss: 2.2469 Acc: 0.0000

Epoch 97/100
----------
train Loss: 2.9705 Acc: 0.0435
val Loss: 2.2953 Acc: 0.0000

Epoch 98/100
----------
train Loss: 2.9623 Acc: 0.0489
val Loss: 2.2189 Acc: 0.0000

Epoch 99/100
----------
train Loss: 2.9781 Acc: 0.0380
val Loss: 2.3541 Acc: 0.0000

Epoch 100/100
----------
train Loss: 2.9656 Acc: 0.0489
val Loss: 2.2218 Acc: 0.0000

Training complete in 20m 44s
Best val Acc: 0.166667
************************* 第 12 折 *************************
train: 181 val: 9
Epoch 1/100
----------
train Loss: 2.9747 Acc: 0.0489
val Loss: 2.2254 Acc: 0.0000

Epoch 2/100
----------
train Loss: 2.9919 Acc: 0.0326
val Loss: 2.2853 Acc: 0.0000

Epoch 3/100
----------
train Loss: 2.9740 Acc: 0.0489
val Loss: 2.2625 Acc: 0.0833

Epoch 4/100
----------
train Loss: 2.9727 Acc: 0.0380
val Loss: 2.2825 Acc: 0.0000

Epoch 5/100
----------
train Loss: 2.9716 Acc: 0.0598
val Loss: 2.1914 Acc: 0.0000

Epoch 6/100
----------
train Loss: 2.9917 Acc: 0.0543
val Loss: 2.3077 Acc: 0.0000

Epoch 7/100
----------
train Loss: 2.9744 Acc: 0.0598
val Loss: 2.2636 Acc: 0.0000

Epoch 8/100
----------
train Loss: 2.9686 Acc: 0.0326
val Loss: 2.1919 Acc: 0.0000

Epoch 9/100
----------
train Loss: 2.9894 Acc: 0.0380
val Loss: 2.2771 Acc: 0.0000

Epoch 10/100
----------
train Loss: 2.9881 Acc: 0.0054
val Loss: 2.1810 Acc: 0.0000

Epoch 11/100
----------
train Loss: 2.9710 Acc: 0.0435
val Loss: 2.2513 Acc: 0.0000

Epoch 12/100
----------
train Loss: 2.9502 Acc: 0.0435
val Loss: 2.2200 Acc: 0.0000

Epoch 13/100
----------
train Loss: 2.9855 Acc: 0.0380
val Loss: 2.2206 Acc: 0.0000

Epoch 14/100
----------
train Loss: 2.9612 Acc: 0.0435
val Loss: 2.2256 Acc: 0.0000

Epoch 15/100
----------
train Loss: 2.9874 Acc: 0.0489
val Loss: 2.2122 Acc: 0.0833

Epoch 16/100
----------
train Loss: 2.9513 Acc: 0.0435
val Loss: 2.1738 Acc: 0.0833

Epoch 17/100
----------
train Loss: 2.9879 Acc: 0.0543
val Loss: 2.1695 Acc: 0.0833

Epoch 18/100
----------
train Loss: 2.9725 Acc: 0.0707
val Loss: 2.1501 Acc: 0.0000

Epoch 19/100
----------
train Loss: 2.9822 Acc: 0.0435
val Loss: 2.1865 Acc: 0.0000

Epoch 20/100
----------
train Loss: 3.0078 Acc: 0.0054
val Loss: 2.2494 Acc: 0.0000

Epoch 21/100
----------
train Loss: 2.9676 Acc: 0.0435
val Loss: 2.2800 Acc: 0.0000

Epoch 22/100
----------
train Loss: 2.9709 Acc: 0.0435
val Loss: 2.1478 Acc: 0.0833

Epoch 23/100
----------
train Loss: 2.9431 Acc: 0.0326
val Loss: 2.2725 Acc: 0.0000

Epoch 24/100
----------
train Loss: 2.9630 Acc: 0.0652
val Loss: 2.2244 Acc: 0.0000

Epoch 25/100
----------
train Loss: 2.9880 Acc: 0.0435
val Loss: 2.2121 Acc: 0.1667

Epoch 26/100
----------
train Loss: 2.9644 Acc: 0.0543
val Loss: 2.2282 Acc: 0.0000

Epoch 27/100
----------
train Loss: 2.9790 Acc: 0.0707
val Loss: 2.3108 Acc: 0.0000

Epoch 28/100
----------
train Loss: 2.9759 Acc: 0.0489
val Loss: 2.1459 Acc: 0.0000

Epoch 29/100
----------
train Loss: 2.9813 Acc: 0.0435
val Loss: 2.2187 Acc: 0.0000

Epoch 30/100
----------
train Loss: 2.9828 Acc: 0.0489
val Loss: 2.1694 Acc: 0.0000

Epoch 31/100
----------
train Loss: 2.9558 Acc: 0.0543
val Loss: 2.1932 Acc: 0.0833

Epoch 32/100
----------
train Loss: 2.9667 Acc: 0.0598
val Loss: 2.3344 Acc: 0.0000

Epoch 33/100
----------
train Loss: 2.9865 Acc: 0.0489
val Loss: 2.2164 Acc: 0.0000

Epoch 34/100
----------
train Loss: 2.9731 Acc: 0.0489
val Loss: 2.1943 Acc: 0.0000

Epoch 35/100
----------
train Loss: 2.9830 Acc: 0.0380
val Loss: 2.2746 Acc: 0.0000

Epoch 36/100
----------
train Loss: 2.9690 Acc: 0.0761
val Loss: 2.2680 Acc: 0.0000

Epoch 37/100
----------
train Loss: 2.9794 Acc: 0.0543
val Loss: 2.1887 Acc: 0.0000

Epoch 38/100
----------
train Loss: 2.9624 Acc: 0.0435
val Loss: 2.3163 Acc: 0.0000

Epoch 39/100
----------
train Loss: 2.9774 Acc: 0.0217
val Loss: 2.2490 Acc: 0.0000

Epoch 40/100
----------
train Loss: 2.9630 Acc: 0.0598
val Loss: 2.2045 Acc: 0.0833

Epoch 41/100
----------
train Loss: 2.9809 Acc: 0.0217
val Loss: 2.2090 Acc: 0.1667

Epoch 42/100
----------
train Loss: 2.9839 Acc: 0.0217
val Loss: 2.1401 Acc: 0.0833

Epoch 43/100
----------
train Loss: 2.9653 Acc: 0.0707
val Loss: 2.2170 Acc: 0.0000

Epoch 44/100
----------
train Loss: 2.9699 Acc: 0.0435
val Loss: 2.1823 Acc: 0.0000

Epoch 45/100
----------
train Loss: 2.9744 Acc: 0.0435
val Loss: 2.1785 Acc: 0.0833

Epoch 46/100
----------
train Loss: 2.9880 Acc: 0.0489
val Loss: 2.1930 Acc: 0.0833

Epoch 47/100
----------
train Loss: 2.9641 Acc: 0.0380
val Loss: 2.2589 Acc: 0.0000

Epoch 48/100
----------
train Loss: 2.9981 Acc: 0.0380
val Loss: 2.2231 Acc: 0.0833

Epoch 49/100
----------
train Loss: 2.9817 Acc: 0.0598
val Loss: 2.2482 Acc: 0.0833

Epoch 50/100
----------
train Loss: 2.9727 Acc: 0.0598
val Loss: 2.2751 Acc: 0.0833

Epoch 51/100
----------
train Loss: 2.9711 Acc: 0.0435
val Loss: 2.2396 Acc: 0.0000

Epoch 52/100
----------
train Loss: 2.9759 Acc: 0.0435
val Loss: 2.1872 Acc: 0.0833

Epoch 53/100
----------
train Loss: 2.9931 Acc: 0.0380
val Loss: 2.2032 Acc: 0.0000

Epoch 54/100
----------
train Loss: 2.9538 Acc: 0.0652
val Loss: 2.1812 Acc: 0.0833

Epoch 55/100
----------
train Loss: 2.9746 Acc: 0.0217
val Loss: 2.2109 Acc: 0.0833

Epoch 56/100
----------
train Loss: 2.9748 Acc: 0.0598
val Loss: 2.1614 Acc: 0.0833

Epoch 57/100
----------
train Loss: 2.9618 Acc: 0.0326
val Loss: 2.2279 Acc: 0.0000

Epoch 58/100
----------
train Loss: 2.9713 Acc: 0.0435
val Loss: 2.2733 Acc: 0.0000

Epoch 59/100
----------
train Loss: 2.9411 Acc: 0.0489
val Loss: 2.3084 Acc: 0.0000

Epoch 60/100
----------
train Loss: 2.9762 Acc: 0.0217
val Loss: 2.1969 Acc: 0.1667

Epoch 61/100
----------
train Loss: 2.9702 Acc: 0.0489
val Loss: 2.2550 Acc: 0.0833

Epoch 62/100
----------
train Loss: 2.9677 Acc: 0.0543
val Loss: 2.2594 Acc: 0.0000

Epoch 63/100
----------
train Loss: 2.9718 Acc: 0.0598
val Loss: 2.2869 Acc: 0.0000

Epoch 64/100
----------
train Loss: 2.9794 Acc: 0.0217
val Loss: 2.2807 Acc: 0.0000

Epoch 65/100
----------
train Loss: 2.9660 Acc: 0.0326
val Loss: 2.2566 Acc: 0.0000

Epoch 66/100
----------
train Loss: 2.9514 Acc: 0.0543
val Loss: 2.2996 Acc: 0.0000

Epoch 67/100
----------
train Loss: 2.9770 Acc: 0.0380
val Loss: 2.3494 Acc: 0.0000

Epoch 68/100
----------
train Loss: 2.9612 Acc: 0.0489
val Loss: 2.0761 Acc: 0.2500

Epoch 69/100
----------
train Loss: 2.9734 Acc: 0.0598
val Loss: 2.2552 Acc: 0.0833

Epoch 70/100
----------
train Loss: 2.9808 Acc: 0.0652
val Loss: 2.2553 Acc: 0.0000

Epoch 71/100
----------
train Loss: 2.9890 Acc: 0.0380
val Loss: 2.1646 Acc: 0.1667

Epoch 72/100
----------
train Loss: 2.9779 Acc: 0.0326
val Loss: 2.2434 Acc: 0.0000

Epoch 73/100
----------
train Loss: 2.9770 Acc: 0.0435
val Loss: 2.2495 Acc: 0.0000

Epoch 74/100
----------
train Loss: 2.9675 Acc: 0.0380
val Loss: 2.1697 Acc: 0.0833

Epoch 75/100
----------
train Loss: 2.9564 Acc: 0.0489
val Loss: 2.1705 Acc: 0.0833

Epoch 76/100
----------
train Loss: 2.9712 Acc: 0.0380
val Loss: 2.1840 Acc: 0.0000

Epoch 77/100
----------
train Loss: 2.9596 Acc: 0.0598
val Loss: 2.3502 Acc: 0.0000

Epoch 78/100
----------
train Loss: 2.9681 Acc: 0.0815
val Loss: 2.1450 Acc: 0.0000

Epoch 79/100
----------
train Loss: 2.9676 Acc: 0.0326
val Loss: 2.3493 Acc: 0.0000

Epoch 80/100
----------
train Loss: 2.9642 Acc: 0.0598
val Loss: 2.2462 Acc: 0.0000

Epoch 81/100
----------
train Loss: 2.9515 Acc: 0.0543
val Loss: 2.3386 Acc: 0.0000

Epoch 82/100
----------
train Loss: 2.9588 Acc: 0.0598
val Loss: 2.2476 Acc: 0.0000

Epoch 83/100
----------
train Loss: 2.9795 Acc: 0.0272
val Loss: 2.1884 Acc: 0.0833

Epoch 84/100
----------
train Loss: 2.9608 Acc: 0.0543
val Loss: 2.2129 Acc: 0.0000

Epoch 85/100
----------
train Loss: 2.9855 Acc: 0.0380
val Loss: 2.2831 Acc: 0.0833

Epoch 86/100
----------
train Loss: 2.9901 Acc: 0.0543
val Loss: 2.2214 Acc: 0.0833

Epoch 87/100
----------
train Loss: 2.9608 Acc: 0.0272
val Loss: 2.2942 Acc: 0.0000

Epoch 88/100
----------
train Loss: 2.9715 Acc: 0.0543
val Loss: 2.2725 Acc: 0.0000

Epoch 89/100
----------
train Loss: 2.9705 Acc: 0.0598
val Loss: 2.2136 Acc: 0.0833

Epoch 90/100
----------
train Loss: 2.9773 Acc: 0.0707
val Loss: 2.1792 Acc: 0.0000

Epoch 91/100
----------
train Loss: 2.9803 Acc: 0.0598
val Loss: 2.1227 Acc: 0.1667

Epoch 92/100
----------
train Loss: 2.9760 Acc: 0.0543
val Loss: 2.2040 Acc: 0.0833

Epoch 93/100
----------
train Loss: 2.9556 Acc: 0.0380
val Loss: 2.3494 Acc: 0.0000

Epoch 94/100
----------
train Loss: 2.9923 Acc: 0.0163
val Loss: 2.2764 Acc: 0.0000

Epoch 95/100
----------
train Loss: 2.9702 Acc: 0.0272
val Loss: 2.2293 Acc: 0.0833

Epoch 96/100
----------
train Loss: 2.9492 Acc: 0.0707
val Loss: 2.1709 Acc: 0.0833

Epoch 97/100
----------
train Loss: 2.9715 Acc: 0.0435
val Loss: 2.2041 Acc: 0.0000

Epoch 98/100
----------
train Loss: 2.9718 Acc: 0.0489
val Loss: 2.2762 Acc: 0.1667

Epoch 99/100
----------
train Loss: 2.9723 Acc: 0.0326
val Loss: 2.2959 Acc: 0.0833

Epoch 100/100
----------
train Loss: 2.9894 Acc: 0.0543
val Loss: 2.2751 Acc: 0.0000

Training complete in 20m 46s
Best val Acc: 0.250000
************************* 第 13 折 *************************
train: 181 val: 9
Epoch 1/100
----------
train Loss: 2.9553 Acc: 0.0543
val Loss: 2.3786 Acc: 0.0000

Epoch 2/100
----------
train Loss: 2.9702 Acc: 0.0435
val Loss: 2.3131 Acc: 0.0000

Epoch 3/100
----------
train Loss: 2.9504 Acc: 0.0543
val Loss: 2.2932 Acc: 0.0833

Epoch 4/100
----------
train Loss: 2.9545 Acc: 0.0380
val Loss: 2.3727 Acc: 0.0000

Epoch 5/100
----------
train Loss: 2.9618 Acc: 0.0326
val Loss: 2.3159 Acc: 0.0833

Epoch 6/100
----------
train Loss: 2.9525 Acc: 0.0489
val Loss: 2.2957 Acc: 0.0000

Epoch 7/100
----------
train Loss: 2.9797 Acc: 0.0815
val Loss: 2.4976 Acc: 0.0000

Epoch 8/100
----------
train Loss: 2.9313 Acc: 0.0380
val Loss: 2.2079 Acc: 0.0000

Epoch 9/100
----------
train Loss: 2.9711 Acc: 0.0435
val Loss: 2.3105 Acc: 0.0000

Epoch 10/100
----------
train Loss: 2.9622 Acc: 0.0489
val Loss: 2.2841 Acc: 0.0833

Epoch 11/100
----------
train Loss: 2.9761 Acc: 0.0272
val Loss: 2.3434 Acc: 0.0000

Epoch 12/100
----------
train Loss: 2.9830 Acc: 0.0435
val Loss: 2.3294 Acc: 0.0833

Epoch 13/100
----------
train Loss: 2.9574 Acc: 0.0380
val Loss: 2.2651 Acc: 0.0000

Epoch 14/100
----------
train Loss: 2.9608 Acc: 0.0598
val Loss: 2.2937 Acc: 0.0000

Epoch 15/100
----------
train Loss: 2.9762 Acc: 0.0489
val Loss: 2.2466 Acc: 0.0833

Epoch 16/100
----------
train Loss: 2.9848 Acc: 0.0380
val Loss: 2.2900 Acc: 0.0000

Epoch 17/100
----------
train Loss: 2.9787 Acc: 0.0272
val Loss: 2.3461 Acc: 0.0000

Epoch 18/100
----------
train Loss: 2.9645 Acc: 0.0435
val Loss: 2.3406 Acc: 0.0000

Epoch 19/100
----------
train Loss: 2.9567 Acc: 0.0489
val Loss: 2.4017 Acc: 0.0000

Epoch 20/100
----------
train Loss: 2.9668 Acc: 0.0543
val Loss: 2.3489 Acc: 0.0000

Epoch 21/100
----------
train Loss: 2.9946 Acc: 0.0380
val Loss: 2.2655 Acc: 0.0000

Epoch 22/100
----------
train Loss: 2.9549 Acc: 0.0598
val Loss: 2.3960 Acc: 0.0000

Epoch 23/100
----------
train Loss: 2.9604 Acc: 0.0380
val Loss: 2.3039 Acc: 0.0833

Epoch 24/100
----------
train Loss: 2.9538 Acc: 0.0652
val Loss: 2.2413 Acc: 0.0000

Epoch 25/100
----------
train Loss: 2.9630 Acc: 0.0543
val Loss: 2.4588 Acc: 0.0000

Epoch 26/100
----------
train Loss: 2.9528 Acc: 0.0489
val Loss: 2.3548 Acc: 0.0000

Epoch 27/100
----------
train Loss: 2.9395 Acc: 0.0543
val Loss: 2.2863 Acc: 0.0000

Epoch 28/100
----------
train Loss: 2.9664 Acc: 0.0489
val Loss: 2.3791 Acc: 0.0000

Epoch 29/100
----------
train Loss: 2.9742 Acc: 0.0435
val Loss: 2.3371 Acc: 0.0000

Epoch 30/100
----------
train Loss: 2.9746 Acc: 0.0543
val Loss: 2.4093 Acc: 0.0000

Epoch 31/100
----------
train Loss: 2.9810 Acc: 0.0326
val Loss: 2.3250 Acc: 0.0833

Epoch 32/100
----------
train Loss: 2.9753 Acc: 0.0380
val Loss: 2.1992 Acc: 0.0833

Epoch 33/100
----------
train Loss: 2.9482 Acc: 0.0598
val Loss: 2.2921 Acc: 0.0000

Epoch 34/100
----------
train Loss: 2.9794 Acc: 0.0489
val Loss: 2.3782 Acc: 0.0000

Epoch 35/100
----------
train Loss: 2.9749 Acc: 0.0326
val Loss: 2.3032 Acc: 0.0833

Epoch 36/100
----------
train Loss: 2.9596 Acc: 0.0543
val Loss: 2.2461 Acc: 0.0000

Epoch 37/100
----------
train Loss: 2.9805 Acc: 0.0109
val Loss: 2.3621 Acc: 0.0833

Epoch 38/100
----------
train Loss: 2.9651 Acc: 0.0489
val Loss: 2.2835 Acc: 0.0000

Epoch 39/100
----------
train Loss: 2.9591 Acc: 0.0598
val Loss: 2.3911 Acc: 0.0833

Epoch 40/100
----------
train Loss: 2.9701 Acc: 0.0435
val Loss: 2.3329 Acc: 0.0000

Epoch 41/100
----------
train Loss: 2.9753 Acc: 0.0489
val Loss: 2.3452 Acc: 0.0000

Epoch 42/100
----------
train Loss: 2.9467 Acc: 0.0543
val Loss: 2.3050 Acc: 0.0000

Epoch 43/100
----------
train Loss: 2.9703 Acc: 0.0598
val Loss: 2.3415 Acc: 0.0000

Epoch 44/100
----------
train Loss: 2.9751 Acc: 0.0272
val Loss: 2.2519 Acc: 0.0000

Epoch 45/100
----------
train Loss: 2.9741 Acc: 0.0598
val Loss: 2.3132 Acc: 0.0000

Epoch 46/100
----------
train Loss: 2.9643 Acc: 0.0326
val Loss: 2.2859 Acc: 0.0000

Epoch 47/100
----------
train Loss: 2.9579 Acc: 0.0272
val Loss: 2.3676 Acc: 0.0000

Epoch 48/100
----------
train Loss: 2.9727 Acc: 0.0435
val Loss: 2.3381 Acc: 0.0000

Epoch 49/100
----------
train Loss: 2.9808 Acc: 0.0489
val Loss: 2.4044 Acc: 0.0000

Epoch 50/100
----------
train Loss: 2.9816 Acc: 0.0380
val Loss: 2.4264 Acc: 0.0000

Epoch 51/100
----------
train Loss: 2.9762 Acc: 0.0543
val Loss: 2.2863 Acc: 0.0000

Epoch 52/100
----------
train Loss: 2.9534 Acc: 0.0598
val Loss: 2.2437 Acc: 0.0000

Epoch 53/100
----------
train Loss: 2.9531 Acc: 0.0707
val Loss: 2.2895 Acc: 0.0833

Epoch 54/100
----------
train Loss: 2.9569 Acc: 0.0707
val Loss: 2.2406 Acc: 0.0833

Epoch 55/100
----------
train Loss: 2.9865 Acc: 0.0489
val Loss: 2.3547 Acc: 0.0000

Epoch 56/100
----------
train Loss: 2.9630 Acc: 0.0326
val Loss: 2.2650 Acc: 0.0000

Epoch 57/100
----------
train Loss: 2.9581 Acc: 0.0326
val Loss: 2.3396 Acc: 0.0000

Epoch 58/100
----------
train Loss: 2.9740 Acc: 0.0272
val Loss: 2.2479 Acc: 0.0833

Epoch 59/100
----------
train Loss: 2.9649 Acc: 0.0272
val Loss: 2.2829 Acc: 0.0000

Epoch 60/100
----------
train Loss: 2.9629 Acc: 0.0326
val Loss: 2.4178 Acc: 0.0000

Epoch 61/100
----------
train Loss: 2.9643 Acc: 0.0652
val Loss: 2.2433 Acc: 0.0833

Epoch 62/100
----------
train Loss: 2.9600 Acc: 0.0707
val Loss: 2.3335 Acc: 0.0000

Epoch 63/100
----------
train Loss: 2.9674 Acc: 0.0815
val Loss: 2.2320 Acc: 0.1667

Epoch 64/100
----------
train Loss: 2.9649 Acc: 0.0489
val Loss: 2.3790 Acc: 0.0000

Epoch 65/100
----------
train Loss: 2.9706 Acc: 0.0543
val Loss: 2.2024 Acc: 0.0833

Epoch 66/100
----------
train Loss: 2.9481 Acc: 0.0870
val Loss: 2.2798 Acc: 0.0833

Epoch 67/100
----------
train Loss: 2.9710 Acc: 0.0435
val Loss: 2.2756 Acc: 0.0833

Epoch 68/100
----------
train Loss: 2.9645 Acc: 0.0272
val Loss: 2.2634 Acc: 0.0833

Epoch 69/100
----------
train Loss: 2.9692 Acc: 0.0489
val Loss: 2.3458 Acc: 0.0000

Epoch 70/100
----------
train Loss: 2.9644 Acc: 0.0380
val Loss: 2.2768 Acc: 0.0833

Epoch 71/100
----------
train Loss: 2.9566 Acc: 0.0543
val Loss: 2.1770 Acc: 0.1667

Epoch 72/100
----------
train Loss: 2.9545 Acc: 0.0598
val Loss: 2.3357 Acc: 0.0000

Epoch 73/100
----------
train Loss: 2.9623 Acc: 0.0435
val Loss: 2.3604 Acc: 0.0000

Epoch 74/100
----------
train Loss: 2.9652 Acc: 0.0489
val Loss: 2.3259 Acc: 0.0000

Epoch 75/100
----------
train Loss: 2.9691 Acc: 0.0163
val Loss: 2.3370 Acc: 0.0000

Epoch 76/100
----------
train Loss: 2.9534 Acc: 0.0652
val Loss: 2.3740 Acc: 0.0000

Epoch 77/100
----------
train Loss: 2.9360 Acc: 0.0761
val Loss: 2.3104 Acc: 0.0000

Epoch 78/100
----------
train Loss: 2.9311 Acc: 0.0652
val Loss: 2.2719 Acc: 0.0000

Epoch 79/100
----------
train Loss: 2.9824 Acc: 0.0543
val Loss: 2.3929 Acc: 0.0833

Epoch 80/100
----------
train Loss: 2.9815 Acc: 0.0435
val Loss: 2.3239 Acc: 0.0833

Epoch 81/100
----------
train Loss: 2.9759 Acc: 0.0435
val Loss: 2.3785 Acc: 0.0000

Epoch 82/100
----------
train Loss: 2.9699 Acc: 0.0380
val Loss: 2.3632 Acc: 0.0833

Epoch 83/100
----------
train Loss: 2.9622 Acc: 0.0380
val Loss: 2.1757 Acc: 0.1667

Epoch 84/100
----------
train Loss: 2.9689 Acc: 0.0489
val Loss: 2.3248 Acc: 0.1667

Epoch 85/100
----------
train Loss: 2.9556 Acc: 0.0543
val Loss: 2.4220 Acc: 0.0000

Epoch 86/100
----------
train Loss: 2.9509 Acc: 0.0761
val Loss: 2.3574 Acc: 0.0000

Epoch 87/100
----------
train Loss: 2.9604 Acc: 0.0380
val Loss: 2.4392 Acc: 0.0000

Epoch 88/100
----------
train Loss: 2.9702 Acc: 0.0272
val Loss: 2.2581 Acc: 0.0000

Epoch 89/100
----------
train Loss: 2.9725 Acc: 0.0326
val Loss: 2.1971 Acc: 0.0000

Epoch 90/100
----------
train Loss: 2.9801 Acc: 0.0652
val Loss: 2.3641 Acc: 0.0833

Epoch 91/100
----------
train Loss: 2.9575 Acc: 0.0543
val Loss: 2.2627 Acc: 0.0000

Epoch 92/100
----------
train Loss: 2.9468 Acc: 0.0435
val Loss: 2.3257 Acc: 0.0000

Epoch 93/100
----------
train Loss: 2.9696 Acc: 0.0598
val Loss: 2.3179 Acc: 0.0000

Epoch 94/100
----------
train Loss: 2.9741 Acc: 0.0435
val Loss: 2.3871 Acc: 0.0000

Epoch 95/100
----------
train Loss: 2.9566 Acc: 0.0380
val Loss: 2.3494 Acc: 0.0000

Epoch 96/100
----------
train Loss: 2.9641 Acc: 0.0326
val Loss: 2.3686 Acc: 0.0000

Epoch 97/100
----------
train Loss: 2.9666 Acc: 0.0326
val Loss: 2.2504 Acc: 0.0833

Epoch 98/100
----------
train Loss: 2.9591 Acc: 0.0489
val Loss: 2.2716 Acc: 0.0000

Epoch 99/100
----------
train Loss: 2.9536 Acc: 0.0652
val Loss: 2.3607 Acc: 0.0000

Epoch 100/100
----------
train Loss: 2.9694 Acc: 0.0380
val Loss: 2.4562 Acc: 0.0000

Training complete in 20m 46s
Best val Acc: 0.166667
************************* 第 14 折 *************************
train: 181 val: 9
Epoch 1/100
----------
train Loss: 2.9908 Acc: 0.0163
val Loss: 2.3196 Acc: 0.0000

Epoch 2/100
----------
train Loss: 2.9636 Acc: 0.0326
val Loss: 2.3032 Acc: 0.0000

Epoch 3/100
----------
train Loss: 2.9515 Acc: 0.0272
val Loss: 2.3693 Acc: 0.0000

Epoch 4/100
----------
train Loss: 2.9780 Acc: 0.0489
val Loss: 2.3245 Acc: 0.0833

Epoch 5/100
----------
train Loss: 2.9738 Acc: 0.0598
val Loss: 2.3945 Acc: 0.0000

Epoch 6/100
----------
train Loss: 2.9816 Acc: 0.0543
val Loss: 2.3099 Acc: 0.0000

Epoch 7/100
----------
train Loss: 2.9614 Acc: 0.0489
val Loss: 2.4047 Acc: 0.0833

Epoch 8/100
----------
train Loss: 2.9579 Acc: 0.0707
val Loss: 2.3769 Acc: 0.0000

Epoch 9/100
----------
train Loss: 2.9740 Acc: 0.0543
val Loss: 2.3353 Acc: 0.0000

Epoch 10/100
----------
train Loss: 2.9765 Acc: 0.0489
val Loss: 2.4308 Acc: 0.0000

Epoch 11/100
----------
train Loss: 2.9489 Acc: 0.0761
val Loss: 2.4265 Acc: 0.0000

Epoch 12/100
----------
train Loss: 2.9765 Acc: 0.0543
val Loss: 2.3657 Acc: 0.0000

Epoch 13/100
----------
train Loss: 2.9430 Acc: 0.0761
val Loss: 2.4311 Acc: 0.0000

Epoch 14/100
----------
train Loss: 2.9716 Acc: 0.0380
val Loss: 2.3802 Acc: 0.0000

Epoch 15/100
----------
train Loss: 2.9828 Acc: 0.0652
val Loss: 2.4129 Acc: 0.0000

Epoch 16/100
----------
train Loss: 2.9624 Acc: 0.0435
val Loss: 2.3725 Acc: 0.0000

Epoch 17/100
----------
train Loss: 2.9650 Acc: 0.0761
val Loss: 2.3135 Acc: 0.0000

Epoch 18/100
----------
train Loss: 2.9659 Acc: 0.0272
val Loss: 2.3305 Acc: 0.0000

Epoch 19/100
----------
train Loss: 2.9667 Acc: 0.0326
val Loss: 2.3172 Acc: 0.0000

Epoch 20/100
----------
train Loss: 2.9547 Acc: 0.0489
val Loss: 2.3603 Acc: 0.0000

Epoch 21/100
----------
train Loss: 2.9488 Acc: 0.0598
val Loss: 2.3202 Acc: 0.0000

Epoch 22/100
----------
train Loss: 2.9449 Acc: 0.0598
val Loss: 2.3182 Acc: 0.0000

Epoch 23/100
----------
train Loss: 2.9580 Acc: 0.0652
val Loss: 2.3977 Acc: 0.0000

Epoch 24/100
----------
train Loss: 2.9461 Acc: 0.0380
val Loss: 2.2969 Acc: 0.0833

Epoch 25/100
----------
train Loss: 2.9563 Acc: 0.0815
val Loss: 2.2734 Acc: 0.0000

Epoch 26/100
----------
train Loss: 2.9466 Acc: 0.0598
val Loss: 2.3980 Acc: 0.0000

Epoch 27/100
----------
train Loss: 2.9526 Acc: 0.0380
val Loss: 2.3720 Acc: 0.0000

Epoch 28/100
----------
train Loss: 2.9681 Acc: 0.0598
val Loss: 2.3007 Acc: 0.0833

Epoch 29/100
----------
train Loss: 2.9686 Acc: 0.0652
val Loss: 2.3843 Acc: 0.0000

Epoch 30/100
----------
train Loss: 2.9688 Acc: 0.0707
val Loss: 2.2637 Acc: 0.0000

Epoch 31/100
----------
train Loss: 2.9647 Acc: 0.0652
val Loss: 2.3701 Acc: 0.0000

Epoch 32/100
----------
train Loss: 2.9480 Acc: 0.0380
val Loss: 2.3245 Acc: 0.0000

Epoch 33/100
----------
train Loss: 2.9829 Acc: 0.0543
val Loss: 2.3333 Acc: 0.0000

Epoch 34/100
----------
train Loss: 2.9629 Acc: 0.0489
val Loss: 2.4065 Acc: 0.0000

Epoch 35/100
----------
train Loss: 2.9774 Acc: 0.0326
val Loss: 2.3690 Acc: 0.0000

Epoch 36/100
----------
train Loss: 2.9599 Acc: 0.0543
val Loss: 2.2934 Acc: 0.0000

Epoch 37/100
----------
train Loss: 2.9838 Acc: 0.0489
val Loss: 2.3619 Acc: 0.0000

Epoch 38/100
----------
train Loss: 2.9362 Acc: 0.0380
val Loss: 2.3833 Acc: 0.0000

Epoch 39/100
----------
train Loss: 2.9844 Acc: 0.0380
val Loss: 2.4106 Acc: 0.0000

Epoch 40/100
----------
train Loss: 2.9388 Acc: 0.0326
val Loss: 2.3618 Acc: 0.0000

Epoch 41/100
----------
train Loss: 2.9622 Acc: 0.0435
val Loss: 2.3954 Acc: 0.0000

Epoch 42/100
----------
train Loss: 2.9557 Acc: 0.0326
val Loss: 2.3134 Acc: 0.0000

Epoch 43/100
----------
train Loss: 2.9730 Acc: 0.0543
val Loss: 2.4326 Acc: 0.0000

Epoch 44/100
----------
train Loss: 2.9311 Acc: 0.0543
val Loss: 2.3637 Acc: 0.0000

Epoch 45/100
----------
train Loss: 2.9639 Acc: 0.0543
val Loss: 2.3838 Acc: 0.0000

Epoch 46/100
----------
train Loss: 2.9631 Acc: 0.0489
val Loss: 2.2915 Acc: 0.0000

Epoch 47/100
----------
train Loss: 2.9736 Acc: 0.0217
val Loss: 2.4014 Acc: 0.0000

Epoch 48/100
----------
train Loss: 2.9420 Acc: 0.0489
val Loss: 2.3321 Acc: 0.0000

Epoch 49/100
----------
train Loss: 2.9469 Acc: 0.0707
val Loss: 2.3447 Acc: 0.0000

Epoch 50/100
----------
train Loss: 2.9621 Acc: 0.0543
val Loss: 2.3946 Acc: 0.0000

Epoch 51/100
----------
train Loss: 2.9741 Acc: 0.0217
val Loss: 2.3585 Acc: 0.0000

Epoch 52/100
----------
train Loss: 2.9530 Acc: 0.0380
val Loss: 2.4278 Acc: 0.0000

Epoch 53/100
----------
train Loss: 2.9595 Acc: 0.0435
val Loss: 2.3474 Acc: 0.0000

Epoch 54/100
----------
train Loss: 2.9567 Acc: 0.0380
val Loss: 2.3662 Acc: 0.0000

Epoch 55/100
----------
train Loss: 2.9715 Acc: 0.0489
val Loss: 2.3780 Acc: 0.0000

Epoch 56/100
----------
train Loss: 2.9638 Acc: 0.0217
val Loss: 2.4113 Acc: 0.0000

Epoch 57/100
----------
train Loss: 2.9713 Acc: 0.0489
val Loss: 2.4069 Acc: 0.0000

Epoch 58/100
----------
train Loss: 2.9495 Acc: 0.0761
val Loss: 2.4353 Acc: 0.0000

Epoch 59/100
----------
train Loss: 2.9817 Acc: 0.0326
val Loss: 2.3732 Acc: 0.0000

Epoch 60/100
----------
train Loss: 2.9460 Acc: 0.0543
val Loss: 2.4589 Acc: 0.0000

Epoch 61/100
----------
train Loss: 2.9559 Acc: 0.0272
val Loss: 2.3786 Acc: 0.0833

Epoch 62/100
----------
train Loss: 2.9645 Acc: 0.0652
val Loss: 2.3517 Acc: 0.0000

Epoch 63/100
----------
train Loss: 2.9893 Acc: 0.0326
val Loss: 2.3708 Acc: 0.0000

Epoch 64/100
----------
train Loss: 2.9582 Acc: 0.0924
val Loss: 2.3554 Acc: 0.0000

Epoch 65/100
----------
train Loss: 2.9672 Acc: 0.0598
val Loss: 2.3364 Acc: 0.0000

Epoch 66/100
----------
train Loss: 2.9638 Acc: 0.0380
val Loss: 2.3763 Acc: 0.0000

Epoch 67/100
----------
train Loss: 2.9634 Acc: 0.0652
val Loss: 2.4106 Acc: 0.0000

Epoch 68/100
----------
train Loss: 2.9652 Acc: 0.0326
val Loss: 2.3578 Acc: 0.0000

Epoch 69/100
----------
train Loss: 2.9846 Acc: 0.0272
val Loss: 2.4268 Acc: 0.0000

Epoch 70/100
----------
train Loss: 2.9582 Acc: 0.0707
val Loss: 2.4013 Acc: 0.0000

Epoch 71/100
----------
train Loss: 2.9648 Acc: 0.0380
val Loss: 2.4247 Acc: 0.0000

Epoch 72/100
----------
train Loss: 2.9446 Acc: 0.0598
val Loss: 2.3071 Acc: 0.0000

Epoch 73/100
----------
train Loss: 2.9469 Acc: 0.0598
val Loss: 2.3385 Acc: 0.0000

Epoch 74/100
----------
train Loss: 2.9730 Acc: 0.0489
val Loss: 2.3568 Acc: 0.0833

Epoch 75/100
----------
train Loss: 2.9635 Acc: 0.0598
val Loss: 2.3859 Acc: 0.0000

Epoch 76/100
----------
train Loss: 2.9754 Acc: 0.0326
val Loss: 2.3732 Acc: 0.0000

Epoch 77/100
----------
train Loss: 2.9679 Acc: 0.0435
val Loss: 2.3366 Acc: 0.0000

Epoch 78/100
----------
train Loss: 2.9620 Acc: 0.0489
val Loss: 2.3882 Acc: 0.0000

Epoch 79/100
----------
train Loss: 2.9789 Acc: 0.0326
val Loss: 2.3293 Acc: 0.0000

Epoch 80/100
----------
train Loss: 2.9598 Acc: 0.0598
val Loss: 2.3246 Acc: 0.0000

Epoch 81/100
----------
train Loss: 2.9489 Acc: 0.0598
val Loss: 2.4132 Acc: 0.0000

Epoch 82/100
----------
train Loss: 2.9778 Acc: 0.0652
val Loss: 2.3750 Acc: 0.0000

Epoch 83/100
----------
train Loss: 2.9567 Acc: 0.0489
val Loss: 2.3469 Acc: 0.0000

Epoch 84/100
----------
train Loss: 2.9551 Acc: 0.0489
val Loss: 2.3854 Acc: 0.0000

Epoch 85/100
----------
train Loss: 2.9587 Acc: 0.0598
val Loss: 2.2720 Acc: 0.0000

Epoch 86/100
----------
train Loss: 2.9527 Acc: 0.0435
val Loss: 2.4316 Acc: 0.0000

Epoch 87/100
----------
train Loss: 2.9753 Acc: 0.0707
val Loss: 2.3779 Acc: 0.0000

Epoch 88/100
----------
train Loss: 2.9600 Acc: 0.0326
val Loss: 2.4142 Acc: 0.0000

Epoch 89/100
----------
train Loss: 2.9683 Acc: 0.0380
val Loss: 2.3663 Acc: 0.0000

Epoch 90/100
----------
train Loss: 2.9483 Acc: 0.0707
val Loss: 2.4568 Acc: 0.0000

Epoch 91/100
----------
train Loss: 2.9284 Acc: 0.0815
val Loss: 2.2915 Acc: 0.0833

Epoch 92/100
----------
train Loss: 2.9853 Acc: 0.0598
val Loss: 2.3675 Acc: 0.0000

Epoch 93/100
----------
train Loss: 2.9567 Acc: 0.0652
val Loss: 2.2861 Acc: 0.0000

Epoch 94/100
----------
train Loss: 2.9822 Acc: 0.0326
val Loss: 2.3616 Acc: 0.0000

Epoch 95/100
----------
train Loss: 2.9659 Acc: 0.0380
val Loss: 2.2918 Acc: 0.0000

Epoch 96/100
----------
train Loss: 2.9609 Acc: 0.0435
val Loss: 2.3210 Acc: 0.0000

Epoch 97/100
----------
train Loss: 2.9672 Acc: 0.0489
val Loss: 2.3801 Acc: 0.0000

Epoch 98/100
----------
train Loss: 2.9659 Acc: 0.0489
val Loss: 2.4158 Acc: 0.0000

Epoch 99/100
----------
train Loss: 2.9600 Acc: 0.0543
val Loss: 2.3484 Acc: 0.0000

Epoch 100/100
----------
train Loss: 2.9695 Acc: 0.0326
val Loss: 2.3442 Acc: 0.0000

Training complete in 20m 47s
Best val Acc: 0.083333
************************* 第 15 折 *************************
train: 181 val: 9
Epoch 1/100
----------
train Loss: 2.9864 Acc: 0.0326
val Loss: 2.0472 Acc: 0.1667

Epoch 2/100
----------
train Loss: 2.9814 Acc: 0.0435
val Loss: 2.1887 Acc: 0.0833

Epoch 3/100
----------
train Loss: 2.9917 Acc: 0.0380
val Loss: 2.2010 Acc: 0.0000

Epoch 4/100
----------
train Loss: 2.9496 Acc: 0.0815
val Loss: 2.2430 Acc: 0.1667

Epoch 5/100
----------
train Loss: 2.9549 Acc: 0.0652
val Loss: 2.1302 Acc: 0.0833

Epoch 6/100
----------
train Loss: 2.9580 Acc: 0.0761
val Loss: 2.1646 Acc: 0.0833

Epoch 7/100
----------
train Loss: 2.9444 Acc: 0.0380
val Loss: 2.1355 Acc: 0.1667

Epoch 8/100
----------
train Loss: 2.9748 Acc: 0.0380
val Loss: 2.1663 Acc: 0.1667

Epoch 9/100
----------
train Loss: 2.9934 Acc: 0.0272
val Loss: 2.1319 Acc: 0.0833

Epoch 10/100
----------
train Loss: 2.9680 Acc: 0.0272
val Loss: 2.2243 Acc: 0.0833

Epoch 11/100
----------
train Loss: 2.9655 Acc: 0.0489
val Loss: 2.2659 Acc: 0.0000

Epoch 12/100
----------
train Loss: 2.9719 Acc: 0.0489
val Loss: 2.2034 Acc: 0.0000

Epoch 13/100
----------
train Loss: 2.9669 Acc: 0.0707
val Loss: 2.1696 Acc: 0.0000

Epoch 14/100
----------
train Loss: 2.9777 Acc: 0.0543
val Loss: 2.1739 Acc: 0.2500

Epoch 15/100
----------
train Loss: 2.9682 Acc: 0.0435
val Loss: 2.2101 Acc: 0.0833

Epoch 16/100
----------
train Loss: 2.9766 Acc: 0.0272
val Loss: 2.2428 Acc: 0.1667

Epoch 17/100
----------
train Loss: 2.9581 Acc: 0.0761
val Loss: 2.1019 Acc: 0.0833

Epoch 18/100
----------
train Loss: 2.9863 Acc: 0.0272
val Loss: 2.1627 Acc: 0.1667

Epoch 19/100
----------
train Loss: 2.9890 Acc: 0.0326
val Loss: 2.2050 Acc: 0.0000

Epoch 20/100
----------
train Loss: 2.9805 Acc: 0.0543
val Loss: 2.2636 Acc: 0.0000

Epoch 21/100
----------
train Loss: 2.9663 Acc: 0.0489
val Loss: 2.1472 Acc: 0.0833

Epoch 22/100
----------
train Loss: 2.9729 Acc: 0.0435
val Loss: 2.2341 Acc: 0.0000

Epoch 23/100
----------
train Loss: 2.9809 Acc: 0.0272
val Loss: 2.2300 Acc: 0.0000

Epoch 24/100
----------
train Loss: 2.9933 Acc: 0.0435
val Loss: 2.2437 Acc: 0.0000

Epoch 25/100
----------
train Loss: 2.9755 Acc: 0.0489
val Loss: 2.0896 Acc: 0.1667

Epoch 26/100
----------
train Loss: 2.9820 Acc: 0.0326
val Loss: 2.1677 Acc: 0.1667

Epoch 27/100
----------
train Loss: 2.9744 Acc: 0.0489
val Loss: 2.2429 Acc: 0.0000

Epoch 28/100
----------
train Loss: 2.9866 Acc: 0.0652
val Loss: 2.1747 Acc: 0.0000

Epoch 29/100
----------
train Loss: 2.9803 Acc: 0.0380
val Loss: 2.1135 Acc: 0.1667

Epoch 30/100
----------
train Loss: 2.9813 Acc: 0.0380
val Loss: 2.1813 Acc: 0.1667

Epoch 31/100
----------
train Loss: 2.9657 Acc: 0.0272
val Loss: 2.1774 Acc: 0.0833

Epoch 32/100
----------
train Loss: 2.9933 Acc: 0.0272
val Loss: 2.1666 Acc: 0.0833

Epoch 33/100
----------
train Loss: 2.9853 Acc: 0.0326
val Loss: 2.2343 Acc: 0.0833

Epoch 34/100
----------
train Loss: 2.9789 Acc: 0.0272
val Loss: 2.1691 Acc: 0.0833

Epoch 35/100
----------
train Loss: 2.9809 Acc: 0.0109
val Loss: 2.1625 Acc: 0.0833

Epoch 36/100
----------
train Loss: 2.9663 Acc: 0.0543
val Loss: 2.1822 Acc: 0.0000

Epoch 37/100
----------
train Loss: 2.9804 Acc: 0.0326
val Loss: 2.1203 Acc: 0.0833

Epoch 38/100
----------
train Loss: 2.9889 Acc: 0.0543
val Loss: 2.0874 Acc: 0.0833

Epoch 39/100
----------
train Loss: 2.9696 Acc: 0.0707
val Loss: 2.2209 Acc: 0.0000

Epoch 40/100
----------
train Loss: 2.9655 Acc: 0.0326
val Loss: 2.1637 Acc: 0.0833

Epoch 41/100
----------
train Loss: 2.9606 Acc: 0.0598
val Loss: 2.1839 Acc: 0.0833

Epoch 42/100
----------
train Loss: 2.9770 Acc: 0.0326
val Loss: 2.1334 Acc: 0.1667

Epoch 43/100
----------
train Loss: 2.9557 Acc: 0.0326
val Loss: 2.1690 Acc: 0.1667

Epoch 44/100
----------
train Loss: 2.9602 Acc: 0.0435
val Loss: 2.1770 Acc: 0.1667

Epoch 45/100
----------
train Loss: 2.9951 Acc: 0.0380
val Loss: 2.2645 Acc: 0.0000

Epoch 46/100
----------
train Loss: 2.9842 Acc: 0.0326
val Loss: 2.2403 Acc: 0.0000

Epoch 47/100
----------
train Loss: 2.9662 Acc: 0.0598
val Loss: 2.1723 Acc: 0.0833

Epoch 48/100
----------
train Loss: 2.9677 Acc: 0.0489
val Loss: 2.1295 Acc: 0.0000

Epoch 49/100
----------
train Loss: 2.9962 Acc: 0.0435
val Loss: 2.2208 Acc: 0.1667

Epoch 50/100
----------
train Loss: 2.9565 Acc: 0.0326
val Loss: 2.1853 Acc: 0.0833

Epoch 51/100
----------
train Loss: 2.9839 Acc: 0.0217
val Loss: 2.2421 Acc: 0.0833

Epoch 52/100
----------
train Loss: 2.9712 Acc: 0.0652
val Loss: 2.1691 Acc: 0.1667

Epoch 53/100
----------
train Loss: 2.9768 Acc: 0.0380
val Loss: 2.1504 Acc: 0.0833

Epoch 54/100
----------
train Loss: 3.0028 Acc: 0.0380
val Loss: 2.1644 Acc: 0.0833

Epoch 55/100
----------
train Loss: 2.9681 Acc: 0.0380
val Loss: 2.1984 Acc: 0.0833

Epoch 56/100
----------
train Loss: 2.9401 Acc: 0.0598
val Loss: 2.2354 Acc: 0.0833

Epoch 57/100
----------
train Loss: 2.9749 Acc: 0.0543
val Loss: 2.1038 Acc: 0.1667

Epoch 58/100
----------
train Loss: 2.9769 Acc: 0.0435
val Loss: 2.1733 Acc: 0.0000

Epoch 59/100
----------
train Loss: 2.9790 Acc: 0.0217
val Loss: 2.2006 Acc: 0.0833

Epoch 60/100
----------
train Loss: 2.9950 Acc: 0.0217
val Loss: 2.1488 Acc: 0.0000

Epoch 61/100
----------
train Loss: 2.9743 Acc: 0.0435
val Loss: 2.2024 Acc: 0.0833

Epoch 62/100
----------
train Loss: 2.9826 Acc: 0.0489
val Loss: 2.1060 Acc: 0.0833

Epoch 63/100
----------
train Loss: 2.9634 Acc: 0.0543
val Loss: 2.2293 Acc: 0.0833

Epoch 64/100
----------
train Loss: 2.9763 Acc: 0.0489
val Loss: 2.1757 Acc: 0.0833

Epoch 65/100
----------
train Loss: 2.9711 Acc: 0.0380
val Loss: 2.1424 Acc: 0.2500

Epoch 66/100
----------
train Loss: 2.9755 Acc: 0.0489
val Loss: 2.1747 Acc: 0.0000

Epoch 67/100
----------
train Loss: 2.9906 Acc: 0.0543
val Loss: 2.1558 Acc: 0.0833

Epoch 68/100
----------
train Loss: 2.9631 Acc: 0.0435
val Loss: 2.1111 Acc: 0.0833

Epoch 69/100
----------
train Loss: 2.9702 Acc: 0.0761
val Loss: 2.2003 Acc: 0.1667

Epoch 70/100
----------
train Loss: 2.9618 Acc: 0.0272
val Loss: 2.2071 Acc: 0.0833

Epoch 71/100
----------
train Loss: 2.9774 Acc: 0.0380
val Loss: 2.3161 Acc: 0.0000

Epoch 72/100
----------
train Loss: 2.9815 Acc: 0.0326
val Loss: 2.1485 Acc: 0.0833

Epoch 73/100
----------
train Loss: 2.9479 Acc: 0.0761
val Loss: 2.1823 Acc: 0.0833

Epoch 74/100
----------
train Loss: 2.9964 Acc: 0.0489
val Loss: 2.2050 Acc: 0.0000

Epoch 75/100
----------
train Loss: 2.9629 Acc: 0.0489
val Loss: 2.1663 Acc: 0.0000

Epoch 76/100
----------
train Loss: 2.9600 Acc: 0.0489
val Loss: 2.1524 Acc: 0.1667

Epoch 77/100
----------
train Loss: 2.9634 Acc: 0.0217
val Loss: 2.2072 Acc: 0.0000

Epoch 78/100
----------
train Loss: 2.9845 Acc: 0.0326
val Loss: 2.2885 Acc: 0.0000

Epoch 79/100
----------
train Loss: 2.9714 Acc: 0.0380
val Loss: 2.2176 Acc: 0.0000

Epoch 80/100
----------
train Loss: 2.9706 Acc: 0.0435
val Loss: 2.2460 Acc: 0.0000

Epoch 81/100
----------
train Loss: 2.9743 Acc: 0.0326
val Loss: 2.1681 Acc: 0.0000

Epoch 82/100
----------
train Loss: 2.9670 Acc: 0.0543
val Loss: 2.1661 Acc: 0.0833

Epoch 83/100
----------
train Loss: 2.9816 Acc: 0.0380
val Loss: 2.1583 Acc: 0.1667

Epoch 84/100
----------
train Loss: 2.9880 Acc: 0.0598
val Loss: 2.2010 Acc: 0.0000

Epoch 85/100
----------
train Loss: 2.9681 Acc: 0.0326
val Loss: 2.1839 Acc: 0.1667

Epoch 86/100
----------
train Loss: 2.9720 Acc: 0.0272
val Loss: 2.1156 Acc: 0.1667

Epoch 87/100
----------
train Loss: 2.9804 Acc: 0.0435
val Loss: 2.1679 Acc: 0.0000

Epoch 88/100
----------
train Loss: 2.9772 Acc: 0.0543
val Loss: 2.2393 Acc: 0.0000

Epoch 89/100
----------
train Loss: 2.9937 Acc: 0.0326
val Loss: 2.2553 Acc: 0.0000

Epoch 90/100
----------
train Loss: 2.9673 Acc: 0.0543
val Loss: 2.2400 Acc: 0.0833

Epoch 91/100
----------
train Loss: 2.9650 Acc: 0.0326
val Loss: 2.2375 Acc: 0.0833

Epoch 92/100
----------
train Loss: 2.9664 Acc: 0.0272
val Loss: 2.3164 Acc: 0.0833

Epoch 93/100
----------
train Loss: 2.9856 Acc: 0.0326
val Loss: 2.2123 Acc: 0.0833

Epoch 94/100
----------
train Loss: 2.9584 Acc: 0.0815
val Loss: 2.1578 Acc: 0.1667

Epoch 95/100
----------
train Loss: 2.9951 Acc: 0.0326
val Loss: 2.1578 Acc: 0.0000

Epoch 96/100
----------
train Loss: 2.9665 Acc: 0.0380
val Loss: 2.2089 Acc: 0.0833

Epoch 97/100
----------
train Loss: 2.9714 Acc: 0.0598
val Loss: 2.1782 Acc: 0.0833

Epoch 98/100
----------
train Loss: 2.9672 Acc: 0.0543
val Loss: 2.1840 Acc: 0.1667

Epoch 99/100
----------
train Loss: 2.9771 Acc: 0.0380
val Loss: 2.2861 Acc: 0.0000

Epoch 100/100
----------
train Loss: 3.0040 Acc: 0.0380
val Loss: 2.1547 Acc: 0.0000

Training complete in 20m 47s
Best val Acc: 0.250000
************************* 第 16 折 *************************
train: 181 val: 9
Epoch 1/100
----------
train Loss: 2.9613 Acc: 0.0543
val Loss: 2.3261 Acc: 0.0833

Epoch 2/100
----------
train Loss: 2.9572 Acc: 0.0272
val Loss: 2.3515 Acc: 0.0000

Epoch 3/100
----------
train Loss: 2.9657 Acc: 0.0380
val Loss: 2.4654 Acc: 0.0000

Epoch 4/100
----------
train Loss: 2.9480 Acc: 0.0435
val Loss: 2.3255 Acc: 0.0000

Epoch 5/100
----------
train Loss: 2.9598 Acc: 0.0598
val Loss: 2.3271 Acc: 0.0000

Epoch 6/100
----------
train Loss: 2.9644 Acc: 0.0380
val Loss: 2.3140 Acc: 0.0833

Epoch 7/100
----------
train Loss: 2.9679 Acc: 0.0272
val Loss: 2.3024 Acc: 0.0000

Epoch 8/100
----------
train Loss: 2.9674 Acc: 0.0435
val Loss: 2.2568 Acc: 0.0000

Epoch 9/100
----------
train Loss: 2.9524 Acc: 0.0489
val Loss: 2.2993 Acc: 0.0000

Epoch 10/100
----------
train Loss: 2.9554 Acc: 0.0652
val Loss: 2.3055 Acc: 0.0000

Epoch 11/100
----------
train Loss: 2.9556 Acc: 0.0652
val Loss: 2.3252 Acc: 0.0000

Epoch 12/100
----------
train Loss: 2.9397 Acc: 0.0707
val Loss: 2.2310 Acc: 0.0000

Epoch 13/100
----------
train Loss: 2.9572 Acc: 0.0435
val Loss: 2.2755 Acc: 0.0833

Epoch 14/100
----------
train Loss: 2.9629 Acc: 0.0326
val Loss: 2.2418 Acc: 0.0833

Epoch 15/100
----------
train Loss: 2.9727 Acc: 0.0489
val Loss: 2.4226 Acc: 0.0833

Epoch 16/100
----------
train Loss: 2.9723 Acc: 0.0217
val Loss: 2.3533 Acc: 0.0833

Epoch 17/100
----------
train Loss: 2.9688 Acc: 0.0707
val Loss: 2.2565 Acc: 0.0000

Epoch 18/100
----------
train Loss: 2.9598 Acc: 0.0435
val Loss: 2.2577 Acc: 0.0000

Epoch 19/100
----------
train Loss: 2.9595 Acc: 0.0380
val Loss: 2.3314 Acc: 0.0833

Epoch 20/100
----------
train Loss: 2.9626 Acc: 0.0489
val Loss: 2.2185 Acc: 0.0833

Epoch 21/100
----------
train Loss: 2.9651 Acc: 0.0380
val Loss: 2.3814 Acc: 0.0000

Epoch 22/100
----------
train Loss: 2.9673 Acc: 0.0380
val Loss: 2.2295 Acc: 0.0833

Epoch 23/100
----------
train Loss: 2.9804 Acc: 0.0598
val Loss: 2.2538 Acc: 0.0833

Epoch 24/100
----------
train Loss: 2.9767 Acc: 0.0598
val Loss: 2.2359 Acc: 0.0833

Epoch 25/100
----------
train Loss: 2.9665 Acc: 0.0217
val Loss: 2.3096 Acc: 0.0000

Epoch 26/100
----------
train Loss: 2.9593 Acc: 0.0489
val Loss: 2.3065 Acc: 0.1667

Epoch 27/100
----------
train Loss: 2.9635 Acc: 0.0489
val Loss: 2.3300 Acc: 0.0000

Epoch 28/100
----------
train Loss: 2.9466 Acc: 0.0707
val Loss: 2.2913 Acc: 0.0000

Epoch 29/100
----------
train Loss: 2.9844 Acc: 0.0380
val Loss: 2.2526 Acc: 0.0833

Epoch 30/100
----------
train Loss: 2.9484 Acc: 0.0543
val Loss: 2.3633 Acc: 0.0000

Epoch 31/100
----------
train Loss: 2.9664 Acc: 0.0707
val Loss: 2.3248 Acc: 0.0833

Epoch 32/100
----------
train Loss: 2.9532 Acc: 0.0598
val Loss: 2.3080 Acc: 0.0000

Epoch 33/100
----------
train Loss: 2.9667 Acc: 0.0380
val Loss: 2.3125 Acc: 0.0000

Epoch 34/100
----------
train Loss: 2.9671 Acc: 0.0489
val Loss: 2.3287 Acc: 0.0000

Epoch 35/100
----------
train Loss: 2.9552 Acc: 0.0598
val Loss: 2.2574 Acc: 0.0000

Epoch 36/100
----------
train Loss: 2.9696 Acc: 0.0326
val Loss: 2.3534 Acc: 0.0000

Epoch 37/100
----------
train Loss: 2.9589 Acc: 0.0380
val Loss: 2.3890 Acc: 0.0833

Epoch 38/100
----------
train Loss: 2.9668 Acc: 0.0380
val Loss: 2.3659 Acc: 0.0833

Epoch 39/100
----------
train Loss: 2.9722 Acc: 0.0489
val Loss: 2.3281 Acc: 0.0000

Epoch 40/100
----------
train Loss: 2.9832 Acc: 0.0163
val Loss: 2.4074 Acc: 0.0000

Epoch 41/100
----------
train Loss: 2.9791 Acc: 0.0380
val Loss: 2.3413 Acc: 0.0833

Epoch 42/100
----------
train Loss: 2.9719 Acc: 0.0054
val Loss: 2.2790 Acc: 0.0833

Epoch 43/100
----------
train Loss: 2.9840 Acc: 0.0598
val Loss: 2.3413 Acc: 0.0000

Epoch 44/100
----------
train Loss: 2.9538 Acc: 0.0598
val Loss: 2.2428 Acc: 0.0833

Epoch 45/100
----------
train Loss: 2.9636 Acc: 0.0598
val Loss: 2.2959 Acc: 0.0000

Epoch 46/100
----------
train Loss: 2.9520 Acc: 0.0870
val Loss: 2.3373 Acc: 0.0000

Epoch 47/100
----------
train Loss: 2.9623 Acc: 0.0543
val Loss: 2.3051 Acc: 0.0833

Epoch 48/100
----------
train Loss: 2.9623 Acc: 0.0489
val Loss: 2.3679 Acc: 0.0000

Epoch 49/100
----------
train Loss: 2.9561 Acc: 0.0326
val Loss: 2.2691 Acc: 0.0833

Epoch 50/100
----------
train Loss: 2.9447 Acc: 0.0707
val Loss: 2.3160 Acc: 0.0000

Epoch 51/100
----------
train Loss: 2.9578 Acc: 0.0707
val Loss: 2.3303 Acc: 0.0000

Epoch 52/100
----------
train Loss: 2.9745 Acc: 0.0489
val Loss: 2.3310 Acc: 0.0000

Epoch 53/100
----------
train Loss: 2.9697 Acc: 0.0380
val Loss: 2.3442 Acc: 0.0000

Epoch 54/100
----------
train Loss: 2.9680 Acc: 0.0326
val Loss: 2.2691 Acc: 0.1667

Epoch 55/100
----------
train Loss: 2.9590 Acc: 0.0326
val Loss: 2.2843 Acc: 0.0833

Epoch 56/100
----------
train Loss: 2.9923 Acc: 0.0272
val Loss: 2.3290 Acc: 0.0000

Epoch 57/100
----------
train Loss: 2.9692 Acc: 0.0326
val Loss: 2.3977 Acc: 0.0000

Epoch 58/100
----------
train Loss: 2.9546 Acc: 0.0598
val Loss: 2.2717 Acc: 0.0833

Epoch 59/100
----------
train Loss: 2.9657 Acc: 0.0489
val Loss: 2.2110 Acc: 0.0833

Epoch 60/100
----------
train Loss: 2.9575 Acc: 0.0543
val Loss: 2.2577 Acc: 0.0000

Epoch 61/100
----------
train Loss: 2.9845 Acc: 0.0435
val Loss: 2.2512 Acc: 0.0000

Epoch 62/100
----------
train Loss: 2.9632 Acc: 0.0543
val Loss: 2.3321 Acc: 0.0000

Epoch 63/100
----------
train Loss: 2.9751 Acc: 0.0435
val Loss: 2.2534 Acc: 0.0000

Epoch 64/100
----------
train Loss: 2.9674 Acc: 0.0326
val Loss: 2.2974 Acc: 0.0833

Epoch 65/100
----------
train Loss: 2.9556 Acc: 0.0598
val Loss: 2.2707 Acc: 0.0000

Epoch 66/100
----------
train Loss: 2.9427 Acc: 0.0543
val Loss: 2.2436 Acc: 0.0833

Epoch 67/100
----------
train Loss: 2.9621 Acc: 0.0435
val Loss: 2.3418 Acc: 0.0000

Epoch 68/100
----------
train Loss: 2.9554 Acc: 0.0543
val Loss: 2.3322 Acc: 0.0000

Epoch 69/100
----------
train Loss: 2.9640 Acc: 0.0326
val Loss: 2.3948 Acc: 0.0000

Epoch 70/100
----------
train Loss: 2.9649 Acc: 0.0272
val Loss: 2.2890 Acc: 0.0000

Epoch 71/100
----------
train Loss: 2.9414 Acc: 0.0761
val Loss: 2.3055 Acc: 0.0000

Epoch 72/100
----------
train Loss: 2.9700 Acc: 0.0435
val Loss: 2.3400 Acc: 0.0000

Epoch 73/100
----------
train Loss: 2.9599 Acc: 0.0707
val Loss: 2.2612 Acc: 0.0000

Epoch 74/100
----------
train Loss: 2.9655 Acc: 0.0380
val Loss: 2.3893 Acc: 0.0000

Epoch 75/100
----------
train Loss: 2.9639 Acc: 0.0326
val Loss: 2.3252 Acc: 0.0000

Epoch 76/100
----------
train Loss: 2.9575 Acc: 0.0598
val Loss: 2.3586 Acc: 0.0000

Epoch 77/100
----------
train Loss: 2.9670 Acc: 0.0217
val Loss: 2.2998 Acc: 0.0833

Epoch 78/100
----------
train Loss: 2.9593 Acc: 0.0761
val Loss: 2.3532 Acc: 0.0833

Epoch 79/100
----------
train Loss: 2.9777 Acc: 0.0380
val Loss: 2.2797 Acc: 0.0833

Epoch 80/100
----------
train Loss: 2.9554 Acc: 0.0543
val Loss: 2.3444 Acc: 0.0000

Epoch 81/100
----------
train Loss: 2.9846 Acc: 0.0435
val Loss: 2.3721 Acc: 0.0833

Epoch 82/100
----------
train Loss: 2.9603 Acc: 0.0380
val Loss: 2.2464 Acc: 0.0000

Epoch 83/100
----------
train Loss: 2.9502 Acc: 0.0435
val Loss: 2.3659 Acc: 0.0000

Epoch 84/100
----------
train Loss: 2.9809 Acc: 0.0326
val Loss: 2.3599 Acc: 0.0000

Epoch 85/100
----------
train Loss: 2.9688 Acc: 0.0326
val Loss: 2.2704 Acc: 0.0833

Epoch 86/100
----------
train Loss: 2.9774 Acc: 0.0272
val Loss: 2.3760 Acc: 0.0000

Epoch 87/100
----------
train Loss: 2.9686 Acc: 0.0598
val Loss: 2.3416 Acc: 0.0000

Epoch 88/100
----------
train Loss: 2.9552 Acc: 0.0598
val Loss: 2.2400 Acc: 0.0833

Epoch 89/100
----------
train Loss: 2.9603 Acc: 0.0326
val Loss: 2.2702 Acc: 0.0833

Epoch 90/100
----------
train Loss: 2.9587 Acc: 0.0543
val Loss: 2.2981 Acc: 0.0833

Epoch 91/100
----------
train Loss: 2.9585 Acc: 0.0652
val Loss: 2.2838 Acc: 0.0000

Epoch 92/100
----------
train Loss: 2.9690 Acc: 0.0598
val Loss: 2.3105 Acc: 0.0000

Epoch 93/100
----------
train Loss: 2.9608 Acc: 0.0489
val Loss: 2.2952 Acc: 0.0000

Epoch 94/100
----------
train Loss: 2.9666 Acc: 0.0543
val Loss: 2.3590 Acc: 0.0000

Epoch 95/100
----------
train Loss: 2.9493 Acc: 0.0707
val Loss: 2.2484 Acc: 0.0833

Epoch 96/100
----------
train Loss: 2.9550 Acc: 0.0598
val Loss: 2.3577 Acc: 0.0000

Epoch 97/100
----------
train Loss: 2.9433 Acc: 0.0489
val Loss: 2.2294 Acc: 0.0000

Epoch 98/100
----------
train Loss: 2.9481 Acc: 0.0543
val Loss: 2.2639 Acc: 0.0000

Epoch 99/100
----------
train Loss: 2.9536 Acc: 0.0326
val Loss: 2.3265 Acc: 0.0833

Epoch 100/100
----------
train Loss: 2.9663 Acc: 0.0435
val Loss: 2.3766 Acc: 0.0000

Training complete in 20m 47s
Best val Acc: 0.166667
************************* 第 17 折 *************************
train: 181 val: 9
Epoch 1/100
----------
train Loss: 2.9789 Acc: 0.0761
val Loss: 2.3992 Acc: 0.0000

Epoch 2/100
----------
train Loss: 2.9731 Acc: 0.0272
val Loss: 2.3387 Acc: 0.0000

Epoch 3/100
----------
train Loss: 2.9780 Acc: 0.0380
val Loss: 2.3557 Acc: 0.0833

Epoch 4/100
----------
train Loss: 2.9851 Acc: 0.0435
val Loss: 2.2937 Acc: 0.0833

Epoch 5/100
----------
train Loss: 2.9781 Acc: 0.0326
val Loss: 2.3502 Acc: 0.0000

Epoch 6/100
----------
train Loss: 2.9566 Acc: 0.0761
val Loss: 2.2767 Acc: 0.0833

Epoch 7/100
----------
train Loss: 2.9791 Acc: 0.0707
val Loss: 2.3139 Acc: 0.2500

Epoch 8/100
----------
train Loss: 2.9514 Acc: 0.0435
val Loss: 2.3486 Acc: 0.0000

Epoch 9/100
----------
train Loss: 2.9677 Acc: 0.0272
val Loss: 2.3359 Acc: 0.0000

Epoch 10/100
----------
train Loss: 2.9687 Acc: 0.0380
val Loss: 2.3158 Acc: 0.0000

Epoch 11/100
----------
train Loss: 2.9527 Acc: 0.0598
val Loss: 2.2571 Acc: 0.0000

Epoch 12/100
----------
train Loss: 2.9747 Acc: 0.0652
val Loss: 2.2882 Acc: 0.0000

Epoch 13/100
----------
train Loss: 2.9502 Acc: 0.0217
val Loss: 2.2845 Acc: 0.0000

Epoch 14/100
----------
train Loss: 2.9711 Acc: 0.0489
val Loss: 2.3594 Acc: 0.0000

Epoch 15/100
----------
train Loss: 2.9690 Acc: 0.0489
val Loss: 2.3513 Acc: 0.0833

Epoch 16/100
----------
train Loss: 2.9786 Acc: 0.0380
val Loss: 2.3914 Acc: 0.0000

Epoch 17/100
----------
train Loss: 2.9686 Acc: 0.0543
val Loss: 2.2840 Acc: 0.0833

Epoch 18/100
----------
train Loss: 2.9524 Acc: 0.0380
val Loss: 2.3599 Acc: 0.0000

Epoch 19/100
----------
train Loss: 2.9422 Acc: 0.0380
val Loss: 2.3562 Acc: 0.0000

Epoch 20/100
----------
train Loss: 2.9606 Acc: 0.0272
val Loss: 2.3560 Acc: 0.0000

Epoch 21/100
----------
train Loss: 2.9770 Acc: 0.0380
val Loss: 2.3797 Acc: 0.0000

Epoch 22/100
----------
train Loss: 2.9661 Acc: 0.0380
val Loss: 2.3879 Acc: 0.0000

Epoch 23/100
----------
train Loss: 2.9752 Acc: 0.0435
val Loss: 2.2718 Acc: 0.0833

Epoch 24/100
----------
train Loss: 2.9598 Acc: 0.0598
val Loss: 2.2908 Acc: 0.0833

Epoch 25/100
----------
train Loss: 2.9771 Acc: 0.0326
val Loss: 2.3434 Acc: 0.0000

Epoch 26/100
----------
train Loss: 2.9615 Acc: 0.0217
val Loss: 2.2744 Acc: 0.0833

Epoch 27/100
----------
train Loss: 2.9731 Acc: 0.0380
val Loss: 2.3380 Acc: 0.0000

Epoch 28/100
----------
train Loss: 2.9450 Acc: 0.0543
val Loss: 2.3437 Acc: 0.0833

Epoch 29/100
----------
train Loss: 2.9545 Acc: 0.0435
val Loss: 2.3308 Acc: 0.0833

Epoch 30/100
----------
train Loss: 2.9343 Acc: 0.0489
val Loss: 2.3396 Acc: 0.0000

Epoch 31/100
----------
train Loss: 2.9710 Acc: 0.0380
val Loss: 2.3257 Acc: 0.0000

Epoch 32/100
----------
train Loss: 2.9697 Acc: 0.0489
val Loss: 2.2788 Acc: 0.0000

Epoch 33/100
----------
train Loss: 2.9654 Acc: 0.0543
val Loss: 2.3192 Acc: 0.0833

Epoch 34/100
----------
train Loss: 2.9752 Acc: 0.0272
val Loss: 2.3523 Acc: 0.0000

Epoch 35/100
----------
train Loss: 2.9527 Acc: 0.0543
val Loss: 2.2592 Acc: 0.0000

Epoch 36/100
----------
train Loss: 2.9574 Acc: 0.0543
val Loss: 2.2539 Acc: 0.0833

Epoch 37/100
----------
train Loss: 2.9655 Acc: 0.0489
val Loss: 2.3466 Acc: 0.0833

Epoch 38/100
----------
train Loss: 2.9504 Acc: 0.0380
val Loss: 2.3534 Acc: 0.0833

Epoch 39/100
----------
train Loss: 2.9429 Acc: 0.0272
val Loss: 2.3556 Acc: 0.0000

Epoch 40/100
----------
train Loss: 2.9618 Acc: 0.0435
val Loss: 2.2271 Acc: 0.0000

Epoch 41/100
----------
train Loss: 2.9676 Acc: 0.0435
val Loss: 2.3483 Acc: 0.0000

Epoch 42/100
----------
train Loss: 2.9812 Acc: 0.0543
val Loss: 2.3243 Acc: 0.0833

Epoch 43/100
----------
train Loss: 2.9673 Acc: 0.0272
val Loss: 2.3913 Acc: 0.0833

Epoch 44/100
----------
train Loss: 2.9650 Acc: 0.0435
val Loss: 2.4034 Acc: 0.0000

Epoch 45/100
----------
train Loss: 2.9457 Acc: 0.0707
val Loss: 2.3135 Acc: 0.0833

Epoch 46/100
----------
train Loss: 2.9702 Acc: 0.0761
val Loss: 2.3166 Acc: 0.0833

Epoch 47/100
----------
train Loss: 2.9580 Acc: 0.0435
val Loss: 2.3124 Acc: 0.0000

Epoch 48/100
----------
train Loss: 2.9599 Acc: 0.0598
val Loss: 2.2639 Acc: 0.0000

Epoch 49/100
----------
train Loss: 2.9645 Acc: 0.0598
val Loss: 2.3237 Acc: 0.0000

Epoch 50/100
----------
train Loss: 2.9553 Acc: 0.0272
val Loss: 2.3295 Acc: 0.0000

Epoch 51/100
----------
train Loss: 2.9270 Acc: 0.0543
val Loss: 2.3935 Acc: 0.0000

Epoch 52/100
----------
train Loss: 2.9589 Acc: 0.0543
val Loss: 2.3560 Acc: 0.0000

Epoch 53/100
----------
train Loss: 2.9575 Acc: 0.0598
val Loss: 2.4022 Acc: 0.0833

Epoch 54/100
----------
train Loss: 2.9592 Acc: 0.0435
val Loss: 2.3021 Acc: 0.0000

Epoch 55/100
----------
train Loss: 2.9495 Acc: 0.0489
val Loss: 2.3116 Acc: 0.0000

Epoch 56/100
----------
train Loss: 2.9685 Acc: 0.0489
val Loss: 2.3840 Acc: 0.0000

Epoch 57/100
----------
train Loss: 2.9604 Acc: 0.0435
val Loss: 2.4010 Acc: 0.0000

Epoch 58/100
----------
train Loss: 2.9616 Acc: 0.0380
val Loss: 2.3220 Acc: 0.0833

Epoch 59/100
----------
train Loss: 2.9753 Acc: 0.0435
val Loss: 2.4795 Acc: 0.0000

Epoch 60/100
----------
train Loss: 2.9641 Acc: 0.0163
val Loss: 2.3646 Acc: 0.0000

Epoch 61/100
----------
train Loss: 2.9684 Acc: 0.0435
val Loss: 2.2948 Acc: 0.0000

Epoch 62/100
----------
train Loss: 2.9840 Acc: 0.0489
val Loss: 2.3421 Acc: 0.0000

Epoch 63/100
----------
train Loss: 2.9802 Acc: 0.0489
val Loss: 2.4078 Acc: 0.0000

Epoch 64/100
----------
train Loss: 2.9768 Acc: 0.0489
val Loss: 2.3934 Acc: 0.0000

Epoch 65/100
----------
train Loss: 2.9616 Acc: 0.0326
val Loss: 2.3316 Acc: 0.0833

Epoch 66/100
----------
train Loss: 2.9577 Acc: 0.0652
val Loss: 2.3718 Acc: 0.0000

Epoch 67/100
----------
train Loss: 2.9650 Acc: 0.0380
val Loss: 2.3475 Acc: 0.0000

Epoch 68/100
----------
train Loss: 2.9656 Acc: 0.0652
val Loss: 2.4081 Acc: 0.0000

Epoch 69/100
----------
train Loss: 2.9612 Acc: 0.0543
val Loss: 2.2915 Acc: 0.1667

Epoch 70/100
----------
train Loss: 2.9663 Acc: 0.0326
val Loss: 2.4943 Acc: 0.0000

Epoch 71/100
----------
train Loss: 2.9723 Acc: 0.0380
val Loss: 2.2787 Acc: 0.0000

Epoch 72/100
----------
train Loss: 2.9434 Acc: 0.0652
val Loss: 2.3846 Acc: 0.0000

Epoch 73/100
----------
train Loss: 2.9598 Acc: 0.0761
val Loss: 2.3854 Acc: 0.0000

Epoch 74/100
----------
train Loss: 2.9545 Acc: 0.0380
val Loss: 2.3856 Acc: 0.0833

Epoch 75/100
----------
train Loss: 2.9740 Acc: 0.0435
val Loss: 2.4066 Acc: 0.0000

Epoch 76/100
----------
train Loss: 2.9712 Acc: 0.0326
val Loss: 2.3215 Acc: 0.0000

Epoch 77/100
----------
train Loss: 2.9706 Acc: 0.0272
val Loss: 2.2747 Acc: 0.0833

Epoch 78/100
----------
train Loss: 2.9429 Acc: 0.0543
val Loss: 2.3115 Acc: 0.0000

Epoch 79/100
----------
train Loss: 2.9558 Acc: 0.0598
val Loss: 2.3380 Acc: 0.0000

Epoch 80/100
----------
train Loss: 2.9852 Acc: 0.0435
val Loss: 2.3243 Acc: 0.0000

Epoch 81/100
----------
train Loss: 2.9473 Acc: 0.0380
val Loss: 2.3359 Acc: 0.0000

Epoch 82/100
----------
train Loss: 2.9569 Acc: 0.0543
val Loss: 2.3366 Acc: 0.0000

Epoch 83/100
----------
train Loss: 2.9621 Acc: 0.0380
val Loss: 2.3655 Acc: 0.0000

Epoch 84/100
----------
train Loss: 2.9624 Acc: 0.0326
val Loss: 2.3710 Acc: 0.0000

Epoch 85/100
----------
train Loss: 2.9716 Acc: 0.0326
val Loss: 2.3156 Acc: 0.0000

Epoch 86/100
----------
train Loss: 2.9721 Acc: 0.0543
val Loss: 2.3781 Acc: 0.0000

Epoch 87/100
----------
train Loss: 2.9763 Acc: 0.0380
val Loss: 2.2685 Acc: 0.0000

Epoch 88/100
----------
train Loss: 2.9612 Acc: 0.0489
val Loss: 2.3171 Acc: 0.0000

Epoch 89/100
----------
train Loss: 2.9750 Acc: 0.0217
val Loss: 2.3540 Acc: 0.0000

Epoch 90/100
----------
train Loss: 2.9655 Acc: 0.0598
val Loss: 2.3100 Acc: 0.0000

Epoch 91/100
----------
train Loss: 2.9630 Acc: 0.0707
val Loss: 2.3324 Acc: 0.0000

Epoch 92/100
----------
train Loss: 2.9748 Acc: 0.0435
val Loss: 2.3049 Acc: 0.0000

Epoch 93/100
----------
train Loss: 2.9598 Acc: 0.0543
val Loss: 2.3764 Acc: 0.0000

Epoch 94/100
----------
train Loss: 2.9441 Acc: 0.0489
val Loss: 2.3026 Acc: 0.0833

Epoch 95/100
----------
train Loss: 2.9741 Acc: 0.0435
val Loss: 2.3381 Acc: 0.0000

Epoch 96/100
----------
train Loss: 2.9478 Acc: 0.0543
val Loss: 2.2819 Acc: 0.0833

Epoch 97/100
----------
train Loss: 2.9504 Acc: 0.0326
val Loss: 2.2242 Acc: 0.0833

Epoch 98/100
----------
train Loss: 2.9608 Acc: 0.0652
val Loss: 2.3385 Acc: 0.0000

Epoch 99/100
----------
train Loss: 2.9598 Acc: 0.0163
val Loss: 2.3653 Acc: 0.0000

Epoch 100/100
----------
train Loss: 2.9884 Acc: 0.0163
val Loss: 2.3487 Acc: 0.0000

Training complete in 20m 48s
Best val Acc: 0.250000
************************* 第 18 折 *************************
train: 181 val: 9
Epoch 1/100
----------
train Loss: 2.9721 Acc: 0.0435
val Loss: 2.2468 Acc: 0.0833

Epoch 2/100
----------
train Loss: 2.9576 Acc: 0.0380
val Loss: 2.3162 Acc: 0.0833

Epoch 3/100
----------
train Loss: 2.9516 Acc: 0.0707
val Loss: 2.2399 Acc: 0.0000

Epoch 4/100
----------
train Loss: 2.9745 Acc: 0.0598
val Loss: 2.2417 Acc: 0.0833

Epoch 5/100
----------
train Loss: 2.9597 Acc: 0.0543
val Loss: 2.4190 Acc: 0.0000

Epoch 6/100
----------
train Loss: 2.9535 Acc: 0.0543
val Loss: 2.2081 Acc: 0.0833

Epoch 7/100
----------
train Loss: 2.9940 Acc: 0.0543
val Loss: 2.2512 Acc: 0.1667

Epoch 8/100
----------
train Loss: 2.9558 Acc: 0.0652
val Loss: 2.3455 Acc: 0.0000

Epoch 9/100
----------
train Loss: 2.9836 Acc: 0.0163
val Loss: 2.2134 Acc: 0.0000

Epoch 10/100
----------
train Loss: 2.9755 Acc: 0.0435
val Loss: 2.2545 Acc: 0.0833

Epoch 11/100
----------
train Loss: 2.9558 Acc: 0.0380
val Loss: 2.2216 Acc: 0.1667

Epoch 12/100
----------
train Loss: 2.9769 Acc: 0.0217
val Loss: 2.1759 Acc: 0.0833

Epoch 13/100
----------
train Loss: 2.9501 Acc: 0.0489
val Loss: 2.3678 Acc: 0.0000

Epoch 14/100
----------
train Loss: 2.9713 Acc: 0.0652
val Loss: 2.2819 Acc: 0.0833

Epoch 15/100
----------
train Loss: 2.9446 Acc: 0.0435
val Loss: 2.1115 Acc: 0.2500

Epoch 16/100
----------
train Loss: 2.9676 Acc: 0.0326
val Loss: 2.3585 Acc: 0.0000

Epoch 17/100
----------
train Loss: 2.9736 Acc: 0.0326
val Loss: 2.2571 Acc: 0.0833

Epoch 18/100
----------
train Loss: 2.9780 Acc: 0.0435
val Loss: 2.3760 Acc: 0.0000

Epoch 19/100
----------
train Loss: 2.9569 Acc: 0.0707
val Loss: 2.2545 Acc: 0.0000

Epoch 20/100
----------
train Loss: 2.9370 Acc: 0.0435
val Loss: 2.3092 Acc: 0.0833

Epoch 21/100
----------
train Loss: 2.9392 Acc: 0.0380
val Loss: 2.3681 Acc: 0.0000

Epoch 22/100
----------
train Loss: 2.9644 Acc: 0.0435
val Loss: 2.2845 Acc: 0.0000

Epoch 23/100
----------
train Loss: 2.9598 Acc: 0.0435
val Loss: 2.3176 Acc: 0.0000

Epoch 24/100
----------
train Loss: 2.9721 Acc: 0.0272
val Loss: 2.3628 Acc: 0.0000

Epoch 25/100
----------
train Loss: 2.9662 Acc: 0.0272
val Loss: 2.2702 Acc: 0.0833

Epoch 26/100
----------
train Loss: 2.9723 Acc: 0.0489
val Loss: 2.2609 Acc: 0.0000

Epoch 27/100
----------
train Loss: 2.9552 Acc: 0.0489
val Loss: 2.2719 Acc: 0.0000

Epoch 28/100
----------
train Loss: 2.9896 Acc: 0.0489
val Loss: 2.3550 Acc: 0.0000

Epoch 29/100
----------
train Loss: 2.9639 Acc: 0.0598
val Loss: 2.2463 Acc: 0.1667

Epoch 30/100
----------
train Loss: 2.9763 Acc: 0.0652
val Loss: 2.3712 Acc: 0.0000

Epoch 31/100
----------
train Loss: 2.9515 Acc: 0.0543
val Loss: 2.3098 Acc: 0.0000

Epoch 32/100
----------
train Loss: 2.9685 Acc: 0.0435
val Loss: 2.2701 Acc: 0.0833

Epoch 33/100
----------
train Loss: 2.9579 Acc: 0.0543
val Loss: 2.3047 Acc: 0.0000

Epoch 34/100
----------
train Loss: 2.9536 Acc: 0.0652
val Loss: 2.2751 Acc: 0.0833

Epoch 35/100
----------
train Loss: 2.9656 Acc: 0.0543
val Loss: 2.2461 Acc: 0.0000

Epoch 36/100
----------
train Loss: 2.9743 Acc: 0.0380
val Loss: 2.2946 Acc: 0.0000

Epoch 37/100
----------
train Loss: 2.9522 Acc: 0.0435
val Loss: 2.3068 Acc: 0.0000

Epoch 38/100
----------
train Loss: 2.9736 Acc: 0.0326
val Loss: 2.1749 Acc: 0.0833

Epoch 39/100
----------
train Loss: 2.9665 Acc: 0.0380
val Loss: 2.2394 Acc: 0.0000

Epoch 40/100
----------
train Loss: 2.9587 Acc: 0.0435
val Loss: 2.3128 Acc: 0.0000

Epoch 41/100
----------
train Loss: 2.9657 Acc: 0.0435
val Loss: 2.2784 Acc: 0.0000

Epoch 42/100
----------
train Loss: 2.9660 Acc: 0.0435
val Loss: 2.2563 Acc: 0.0000

Epoch 43/100
----------
train Loss: 2.9761 Acc: 0.0598
val Loss: 2.3140 Acc: 0.0833

Epoch 44/100
----------
train Loss: 2.9583 Acc: 0.0543
val Loss: 2.2666 Acc: 0.0833

Epoch 45/100
----------
train Loss: 2.9792 Acc: 0.0435
val Loss: 2.2755 Acc: 0.0000

Epoch 46/100
----------
train Loss: 2.9609 Acc: 0.0652
val Loss: 2.2578 Acc: 0.0833

Epoch 47/100
----------
train Loss: 2.9814 Acc: 0.0543
val Loss: 2.1964 Acc: 0.0000

Epoch 48/100
----------
train Loss: 2.9497 Acc: 0.0489
val Loss: 2.3252 Acc: 0.0000

Epoch 49/100
----------
train Loss: 2.9507 Acc: 0.0543
val Loss: 2.2403 Acc: 0.0833

Epoch 50/100
----------
train Loss: 2.9784 Acc: 0.0326
val Loss: 2.1802 Acc: 0.0833

Epoch 51/100
----------
train Loss: 2.9797 Acc: 0.0543
val Loss: 2.3509 Acc: 0.0000

Epoch 52/100
----------
train Loss: 2.9570 Acc: 0.0217
val Loss: 2.3678 Acc: 0.0000

Epoch 53/100
----------
train Loss: 2.9769 Acc: 0.0435
val Loss: 2.2125 Acc: 0.0000

Epoch 54/100
----------
train Loss: 2.9627 Acc: 0.0435
val Loss: 2.3950 Acc: 0.0833

Epoch 55/100
----------
train Loss: 2.9660 Acc: 0.0380
val Loss: 2.3328 Acc: 0.0000

Epoch 56/100
----------
train Loss: 2.9856 Acc: 0.0435
val Loss: 2.3108 Acc: 0.1667

Epoch 57/100
----------
train Loss: 2.9585 Acc: 0.0652
val Loss: 2.2504 Acc: 0.1667

Epoch 58/100
----------
train Loss: 2.9838 Acc: 0.0543
val Loss: 2.3389 Acc: 0.0000

Epoch 59/100
----------
train Loss: 2.9543 Acc: 0.0489
val Loss: 2.2501 Acc: 0.0833

Epoch 60/100
----------
train Loss: 2.9594 Acc: 0.0272
val Loss: 2.2714 Acc: 0.0000

Epoch 61/100
----------
train Loss: 2.9777 Acc: 0.0489
val Loss: 2.3222 Acc: 0.0000

Epoch 62/100
----------
train Loss: 2.9831 Acc: 0.0761
val Loss: 2.3577 Acc: 0.0000

Epoch 63/100
----------
train Loss: 2.9862 Acc: 0.0272
val Loss: 2.3460 Acc: 0.0000

Epoch 64/100
----------
train Loss: 2.9624 Acc: 0.0489
val Loss: 2.3477 Acc: 0.0000

Epoch 65/100
----------
train Loss: 2.9543 Acc: 0.0652
val Loss: 2.3262 Acc: 0.0000

Epoch 66/100
----------
train Loss: 2.9653 Acc: 0.0380
val Loss: 2.2530 Acc: 0.0833

Epoch 67/100
----------
train Loss: 2.9787 Acc: 0.0489
val Loss: 2.2517 Acc: 0.0833

Epoch 68/100
----------
train Loss: 2.9723 Acc: 0.0543
val Loss: 2.3795 Acc: 0.0000

Epoch 69/100
----------
train Loss: 2.9587 Acc: 0.0380
val Loss: 2.1981 Acc: 0.0000

Epoch 70/100
----------
train Loss: 2.9638 Acc: 0.0217
val Loss: 2.2726 Acc: 0.0833

Epoch 71/100
----------
train Loss: 2.9572 Acc: 0.0326
val Loss: 2.2881 Acc: 0.0833

Epoch 72/100
----------
train Loss: 2.9649 Acc: 0.0543
val Loss: 2.2851 Acc: 0.0000

Epoch 73/100
----------
train Loss: 2.9667 Acc: 0.0380
val Loss: 2.2867 Acc: 0.0000

Epoch 74/100
----------
train Loss: 2.9615 Acc: 0.0489
val Loss: 2.2425 Acc: 0.0000

Epoch 75/100
----------
train Loss: 2.9630 Acc: 0.0217
val Loss: 2.3478 Acc: 0.0833

Epoch 76/100
----------
train Loss: 2.9551 Acc: 0.0652
val Loss: 2.2528 Acc: 0.0833

Epoch 77/100
----------
train Loss: 2.9700 Acc: 0.0652
val Loss: 2.2132 Acc: 0.0833

Epoch 78/100
----------
train Loss: 2.9761 Acc: 0.0598
val Loss: 2.2824 Acc: 0.0000

Epoch 79/100
----------
train Loss: 2.9581 Acc: 0.0598
val Loss: 2.3633 Acc: 0.0000

Epoch 80/100
----------
train Loss: 2.9855 Acc: 0.0489
val Loss: 2.2347 Acc: 0.0833

Epoch 81/100
----------
train Loss: 2.9693 Acc: 0.0435
val Loss: 2.3363 Acc: 0.0000

Epoch 82/100
----------
train Loss: 2.9742 Acc: 0.0598
val Loss: 2.2737 Acc: 0.0000

Epoch 83/100
----------
train Loss: 2.9533 Acc: 0.0435
val Loss: 2.2329 Acc: 0.1667

Epoch 84/100
----------
train Loss: 2.9696 Acc: 0.0435
val Loss: 2.2118 Acc: 0.0000

Epoch 85/100
----------
train Loss: 2.9659 Acc: 0.0543
val Loss: 2.3644 Acc: 0.0000

Epoch 86/100
----------
train Loss: 2.9631 Acc: 0.0380
val Loss: 2.1945 Acc: 0.0000

Epoch 87/100
----------
train Loss: 2.9536 Acc: 0.0652
val Loss: 2.2788 Acc: 0.0000

Epoch 88/100
----------
train Loss: 2.9862 Acc: 0.0326
val Loss: 2.2155 Acc: 0.0000

Epoch 89/100
----------
train Loss: 2.9591 Acc: 0.0489
val Loss: 2.3245 Acc: 0.0000

Epoch 90/100
----------
train Loss: 2.9759 Acc: 0.0489
val Loss: 2.2399 Acc: 0.0833

Epoch 91/100
----------
train Loss: 2.9652 Acc: 0.0489
val Loss: 2.3539 Acc: 0.0000

Epoch 92/100
----------
train Loss: 2.9635 Acc: 0.0489
val Loss: 2.2821 Acc: 0.0000

Epoch 93/100
----------
train Loss: 2.9704 Acc: 0.0380
val Loss: 2.3299 Acc: 0.0000

Epoch 94/100
----------
train Loss: 2.9792 Acc: 0.0217
val Loss: 2.2824 Acc: 0.0000

Epoch 95/100
----------
train Loss: 2.9645 Acc: 0.0435
val Loss: 2.1838 Acc: 0.0833

Epoch 96/100
----------
train Loss: 2.9529 Acc: 0.0761
val Loss: 2.3124 Acc: 0.0833

Epoch 97/100
----------
train Loss: 2.9609 Acc: 0.0217
val Loss: 2.2521 Acc: 0.0000

Epoch 98/100
----------
train Loss: 2.9786 Acc: 0.0489
val Loss: 2.2521 Acc: 0.0000

Epoch 99/100
----------
train Loss: 2.9565 Acc: 0.0272
val Loss: 2.2083 Acc: 0.1667

Epoch 100/100
----------
train Loss: 2.9630 Acc: 0.0489
val Loss: 2.2191 Acc: 0.0833

Training complete in 20m 50s
Best val Acc: 0.250000
************************* 第 19 折 *************************
train: 181 val: 9
Epoch 1/100
----------
train Loss: 2.9548 Acc: 0.0380
val Loss: 2.2641 Acc: 0.0000

Epoch 2/100
----------
train Loss: 2.9539 Acc: 0.0435
val Loss: 2.2766 Acc: 0.0000

Epoch 3/100
----------
train Loss: 2.9563 Acc: 0.0761
val Loss: 2.1853 Acc: 0.1667

Epoch 4/100
----------
train Loss: 2.9637 Acc: 0.0707
val Loss: 2.1306 Acc: 0.0833

Epoch 5/100
----------
train Loss: 2.9916 Acc: 0.0543
val Loss: 2.2160 Acc: 0.0833

Epoch 6/100
----------
train Loss: 2.9710 Acc: 0.0326
val Loss: 2.1589 Acc: 0.0000

Epoch 7/100
----------
train Loss: 2.9707 Acc: 0.0652
val Loss: 2.2285 Acc: 0.0000

Epoch 8/100
----------
train Loss: 2.9903 Acc: 0.0163
val Loss: 2.1895 Acc: 0.0833

Epoch 9/100
----------
train Loss: 2.9548 Acc: 0.0598
val Loss: 2.0977 Acc: 0.0000

Epoch 10/100
----------
train Loss: 2.9525 Acc: 0.0543
val Loss: 2.1621 Acc: 0.0833

Epoch 11/100
----------
train Loss: 2.9579 Acc: 0.0707
val Loss: 2.2050 Acc: 0.0000

Epoch 12/100
----------
train Loss: 2.9620 Acc: 0.0652
val Loss: 2.2123 Acc: 0.0833

Epoch 13/100
----------
train Loss: 2.9629 Acc: 0.0707
val Loss: 2.1659 Acc: 0.0000

Epoch 14/100
----------
train Loss: 2.9880 Acc: 0.0435
val Loss: 2.2458 Acc: 0.0000

Epoch 15/100
----------
train Loss: 2.9524 Acc: 0.0598
val Loss: 2.1980 Acc: 0.0833

Epoch 16/100
----------
train Loss: 2.9696 Acc: 0.0380
val Loss: 2.1907 Acc: 0.0000

Epoch 17/100
----------
train Loss: 2.9508 Acc: 0.0652
val Loss: 2.2828 Acc: 0.0833

Epoch 18/100
----------
train Loss: 2.9451 Acc: 0.0543
val Loss: 2.2864 Acc: 0.0000

Epoch 19/100
----------
train Loss: 2.9707 Acc: 0.0380
val Loss: 2.1662 Acc: 0.0833

Epoch 20/100
----------
train Loss: 2.9635 Acc: 0.0380
val Loss: 2.3442 Acc: 0.0000

Epoch 21/100
----------
train Loss: 2.9785 Acc: 0.0326
val Loss: 2.1863 Acc: 0.0833

Epoch 22/100
----------
train Loss: 2.9510 Acc: 0.0652
val Loss: 2.2973 Acc: 0.0000

Epoch 23/100
----------
train Loss: 2.9736 Acc: 0.0815
val Loss: 2.2505 Acc: 0.0000

Epoch 24/100
----------
train Loss: 2.9715 Acc: 0.0652
val Loss: 2.2615 Acc: 0.0000

Epoch 25/100
----------
train Loss: 2.9864 Acc: 0.0380
val Loss: 2.3980 Acc: 0.0000

Epoch 26/100
----------
train Loss: 2.9653 Acc: 0.0652
val Loss: 2.3046 Acc: 0.0000

Epoch 27/100
----------
train Loss: 2.9746 Acc: 0.0435
val Loss: 2.3340 Acc: 0.0000

Epoch 28/100
----------
train Loss: 2.9823 Acc: 0.0598
val Loss: 2.2184 Acc: 0.0833

Epoch 29/100
----------
train Loss: 2.9681 Acc: 0.0707
val Loss: 2.1643 Acc: 0.0000

Epoch 30/100
----------
train Loss: 2.9464 Acc: 0.0652
val Loss: 2.2785 Acc: 0.0000

Epoch 31/100
----------
train Loss: 2.9658 Acc: 0.0380
val Loss: 2.2058 Acc: 0.0833

Epoch 32/100
----------
train Loss: 2.9720 Acc: 0.0380
val Loss: 2.2542 Acc: 0.0000

Epoch 33/100
----------
train Loss: 2.9757 Acc: 0.0272
val Loss: 2.1937 Acc: 0.0000

Epoch 34/100
----------
train Loss: 2.9662 Acc: 0.0435
val Loss: 2.2042 Acc: 0.0833

Epoch 35/100
----------
train Loss: 2.9598 Acc: 0.0435
val Loss: 2.2968 Acc: 0.0000

Epoch 36/100
----------
train Loss: 2.9571 Acc: 0.0543
val Loss: 2.2592 Acc: 0.0000

Epoch 37/100
----------
train Loss: 2.9692 Acc: 0.0217
val Loss: 2.3134 Acc: 0.0000

Epoch 38/100
----------
train Loss: 2.9671 Acc: 0.0652
val Loss: 2.2719 Acc: 0.0000

Epoch 39/100
----------
train Loss: 2.9736 Acc: 0.0489
val Loss: 2.2133 Acc: 0.0000

Epoch 40/100
----------
train Loss: 2.9495 Acc: 0.0543
val Loss: 2.2525 Acc: 0.0000

Epoch 41/100
----------
train Loss: 2.9784 Acc: 0.0380
val Loss: 2.2319 Acc: 0.1667

Epoch 42/100
----------
train Loss: 2.9804 Acc: 0.0380
val Loss: 2.2701 Acc: 0.0000

Epoch 43/100
----------
train Loss: 2.9487 Acc: 0.0652
val Loss: 2.1680 Acc: 0.0833

Epoch 44/100
----------
train Loss: 2.9902 Acc: 0.0489
val Loss: 2.2015 Acc: 0.0000

Epoch 45/100
----------
train Loss: 2.9557 Acc: 0.0543
val Loss: 2.2727 Acc: 0.0000

Epoch 46/100
----------
train Loss: 2.9754 Acc: 0.0435
val Loss: 2.3040 Acc: 0.0000

Epoch 47/100
----------
train Loss: 2.9736 Acc: 0.0380
val Loss: 2.1750 Acc: 0.0000

Epoch 48/100
----------
train Loss: 2.9738 Acc: 0.0435
val Loss: 2.2126 Acc: 0.0833

Epoch 49/100
----------
train Loss: 2.9740 Acc: 0.0707
val Loss: 2.2619 Acc: 0.0000

Epoch 50/100
----------
train Loss: 2.9771 Acc: 0.0380
val Loss: 2.2472 Acc: 0.0833

Epoch 51/100
----------
train Loss: 2.9811 Acc: 0.0272
val Loss: 2.2806 Acc: 0.0000

Epoch 52/100
----------
train Loss: 2.9718 Acc: 0.0380
val Loss: 2.2953 Acc: 0.0000

Epoch 53/100
----------
train Loss: 2.9907 Acc: 0.0435
val Loss: 2.1525 Acc: 0.0000

Epoch 54/100
----------
train Loss: 2.9736 Acc: 0.0272
val Loss: 2.2437 Acc: 0.1667

Epoch 55/100
----------
train Loss: 2.9837 Acc: 0.0380
val Loss: 2.2441 Acc: 0.0000

Epoch 56/100
----------
train Loss: 2.9818 Acc: 0.0598
val Loss: 2.1280 Acc: 0.0000

Epoch 57/100
----------
train Loss: 2.9705 Acc: 0.0598
val Loss: 2.1672 Acc: 0.1667

Epoch 58/100
----------
train Loss: 2.9551 Acc: 0.0652
val Loss: 2.3092 Acc: 0.0833

Epoch 59/100
----------
train Loss: 2.9753 Acc: 0.0217
val Loss: 2.1454 Acc: 0.0000

Epoch 60/100
----------
train Loss: 2.9740 Acc: 0.0489
val Loss: 2.2204 Acc: 0.0833

Epoch 61/100
----------
train Loss: 2.9734 Acc: 0.0598
val Loss: 2.2926 Acc: 0.0000

Epoch 62/100
----------
train Loss: 2.9930 Acc: 0.0326
val Loss: 2.2670 Acc: 0.0000

Epoch 63/100
----------
train Loss: 2.9715 Acc: 0.0489
val Loss: 2.2484 Acc: 0.0000

Epoch 64/100
----------
train Loss: 2.9953 Acc: 0.0217
val Loss: 2.1611 Acc: 0.0833

Epoch 65/100
----------
train Loss: 2.9663 Acc: 0.0489
val Loss: 2.2588 Acc: 0.0000

Epoch 66/100
----------
train Loss: 3.0013 Acc: 0.0435
val Loss: 2.2469 Acc: 0.0000

Epoch 67/100
----------
train Loss: 2.9540 Acc: 0.0652
val Loss: 2.2237 Acc: 0.0000

Epoch 68/100
----------
train Loss: 2.9829 Acc: 0.0326
val Loss: 2.2637 Acc: 0.0000

Epoch 69/100
----------
train Loss: 2.9572 Acc: 0.0543
val Loss: 2.1978 Acc: 0.0000

Epoch 70/100
----------
train Loss: 2.9792 Acc: 0.0272
val Loss: 2.2194 Acc: 0.0833

Epoch 71/100
----------
train Loss: 2.9662 Acc: 0.0435
val Loss: 2.2092 Acc: 0.0000

Epoch 72/100
----------
train Loss: 2.9838 Acc: 0.0380
val Loss: 2.2247 Acc: 0.0000

Epoch 73/100
----------
train Loss: 2.9782 Acc: 0.0435
val Loss: 2.2978 Acc: 0.0000

Epoch 74/100
----------
train Loss: 2.9633 Acc: 0.0543
val Loss: 2.2349 Acc: 0.0000

Epoch 75/100
----------
train Loss: 2.9593 Acc: 0.0761
val Loss: 2.1594 Acc: 0.0833

Epoch 76/100
----------
train Loss: 3.0034 Acc: 0.0435
val Loss: 2.2819 Acc: 0.0000

Epoch 77/100
----------
train Loss: 2.9872 Acc: 0.0163
val Loss: 2.2597 Acc: 0.0833

Epoch 78/100
----------
train Loss: 2.9781 Acc: 0.0435
val Loss: 2.2028 Acc: 0.0000

Epoch 79/100
----------
train Loss: 2.9634 Acc: 0.0543
val Loss: 2.3171 Acc: 0.0000

Epoch 80/100
----------
train Loss: 2.9955 Acc: 0.0272
val Loss: 2.2168 Acc: 0.0000

Epoch 81/100
----------
train Loss: 2.9578 Acc: 0.0326
val Loss: 2.1908 Acc: 0.0000

Epoch 82/100
----------
train Loss: 2.9586 Acc: 0.0652
val Loss: 2.3383 Acc: 0.0000

Epoch 83/100
----------
train Loss: 2.9749 Acc: 0.0326
val Loss: 2.1743 Acc: 0.0833

Epoch 84/100
----------
train Loss: 2.9550 Acc: 0.0380
val Loss: 2.2717 Acc: 0.0833

Epoch 85/100
----------
train Loss: 2.9789 Acc: 0.0435
val Loss: 2.2271 Acc: 0.0000

Epoch 86/100
----------
train Loss: 2.9700 Acc: 0.0489
val Loss: 2.1956 Acc: 0.0833

Epoch 87/100
----------
train Loss: 2.9777 Acc: 0.0380
val Loss: 2.2041 Acc: 0.0000

Epoch 88/100
----------
train Loss: 2.9811 Acc: 0.0272
val Loss: 2.2466 Acc: 0.0000

Epoch 89/100
----------
train Loss: 2.9546 Acc: 0.0543
val Loss: 2.2749 Acc: 0.0000

Epoch 90/100
----------
train Loss: 2.9634 Acc: 0.0380
val Loss: 2.2373 Acc: 0.0833

Epoch 91/100
----------
train Loss: 2.9737 Acc: 0.0435
val Loss: 2.1977 Acc: 0.0833

Epoch 92/100
----------
train Loss: 2.9753 Acc: 0.0435
val Loss: 2.3058 Acc: 0.0000

Epoch 93/100
----------
train Loss: 2.9550 Acc: 0.0435
val Loss: 2.3071 Acc: 0.0000

Epoch 94/100
----------
train Loss: 2.9711 Acc: 0.0761
val Loss: 2.2665 Acc: 0.0000

Epoch 95/100
----------
train Loss: 2.9619 Acc: 0.0380
val Loss: 2.2979 Acc: 0.0000

Epoch 96/100
----------
train Loss: 2.9577 Acc: 0.0326
val Loss: 2.2113 Acc: 0.0000

Epoch 97/100
----------
train Loss: 2.9759 Acc: 0.0652
val Loss: 2.1231 Acc: 0.2500

Epoch 98/100
----------
train Loss: 2.9688 Acc: 0.0598
val Loss: 2.2640 Acc: 0.0000

Epoch 99/100
----------
train Loss: 2.9380 Acc: 0.0598
val Loss: 2.2596 Acc: 0.0000

Epoch 100/100
----------
train Loss: 2.9725 Acc: 0.0326
val Loss: 2.2510 Acc: 0.0000

Training complete in 20m 47s
Best val Acc: 0.250000
************************* 第 20 折 *************************
train: 181 val: 9
Epoch 1/100
----------
train Loss: 2.9633 Acc: 0.0543
val Loss: 2.2768 Acc: 0.0833

Epoch 2/100
----------
train Loss: 2.9740 Acc: 0.0435
val Loss: 2.2128 Acc: 0.0833

Epoch 3/100
----------
train Loss: 2.9645 Acc: 0.0380
val Loss: 2.2616 Acc: 0.0000

Epoch 4/100
----------
train Loss: 2.9640 Acc: 0.0489
val Loss: 2.2488 Acc: 0.0000

Epoch 5/100
----------
train Loss: 2.9592 Acc: 0.0598
val Loss: 2.2823 Acc: 0.0000

Epoch 6/100
----------
train Loss: 2.9554 Acc: 0.0489
val Loss: 2.3385 Acc: 0.0833

Epoch 7/100
----------
train Loss: 2.9568 Acc: 0.0543
val Loss: 2.3563 Acc: 0.0000

Epoch 8/100
----------
train Loss: 2.9683 Acc: 0.0435
val Loss: 2.2788 Acc: 0.0000

Epoch 9/100
----------
train Loss: 2.9754 Acc: 0.0435
val Loss: 2.2984 Acc: 0.0000

Epoch 10/100
----------
train Loss: 2.9713 Acc: 0.0652
val Loss: 2.4194 Acc: 0.0833

Epoch 11/100
----------
train Loss: 2.9675 Acc: 0.0435
val Loss: 2.3319 Acc: 0.0000

Epoch 12/100
----------
train Loss: 2.9804 Acc: 0.0489
val Loss: 2.3045 Acc: 0.0000

Epoch 13/100
----------
train Loss: 2.9724 Acc: 0.0543
val Loss: 2.3688 Acc: 0.0000

Epoch 14/100
----------
train Loss: 2.9560 Acc: 0.0380
val Loss: 2.3570 Acc: 0.0000

Epoch 15/100
----------
train Loss: 2.9906 Acc: 0.0435
val Loss: 2.2870 Acc: 0.0000

Epoch 16/100
----------
train Loss: 2.9510 Acc: 0.0652
val Loss: 2.2560 Acc: 0.0000

Epoch 17/100
----------
train Loss: 2.9699 Acc: 0.0489
val Loss: 2.2799 Acc: 0.0833

Epoch 18/100
----------
train Loss: 2.9855 Acc: 0.0163
val Loss: 2.2652 Acc: 0.0833

Epoch 19/100
----------
train Loss: 2.9811 Acc: 0.0652
val Loss: 2.2553 Acc: 0.0000

Epoch 20/100
----------
train Loss: 2.9643 Acc: 0.0652
val Loss: 2.2951 Acc: 0.0000

Epoch 21/100
----------
train Loss: 2.9859 Acc: 0.0380
val Loss: 2.2791 Acc: 0.0000

Epoch 22/100
----------
train Loss: 2.9564 Acc: 0.0543
val Loss: 2.1761 Acc: 0.0000

Epoch 23/100
----------
train Loss: 2.9555 Acc: 0.0489
val Loss: 2.2372 Acc: 0.0000

Epoch 24/100
----------
train Loss: 2.9448 Acc: 0.0326
val Loss: 2.3358 Acc: 0.0000

Epoch 25/100
----------
train Loss: 2.9669 Acc: 0.0707
val Loss: 2.3529 Acc: 0.0000

Epoch 26/100
----------
train Loss: 2.9523 Acc: 0.0598
val Loss: 2.3400 Acc: 0.0000

Epoch 27/100
----------
train Loss: 2.9623 Acc: 0.0435
val Loss: 2.2765 Acc: 0.0000

Epoch 28/100
----------
train Loss: 2.9556 Acc: 0.0489
val Loss: 2.3833 Acc: 0.0000

Epoch 29/100
----------
train Loss: 2.9663 Acc: 0.0217
val Loss: 2.3826 Acc: 0.0000

Epoch 30/100
----------
train Loss: 2.9711 Acc: 0.0489
val Loss: 2.3009 Acc: 0.0833

Epoch 31/100
----------
train Loss: 2.9537 Acc: 0.0326
val Loss: 2.4029 Acc: 0.0000

Epoch 32/100
----------
train Loss: 2.9445 Acc: 0.0707
val Loss: 2.2661 Acc: 0.0833

Epoch 33/100
----------
train Loss: 2.9747 Acc: 0.0435
val Loss: 2.1616 Acc: 0.1667

Epoch 34/100
----------
train Loss: 2.9781 Acc: 0.0435
val Loss: 2.2368 Acc: 0.0833

Epoch 35/100
----------
train Loss: 2.9375 Acc: 0.0652
val Loss: 2.3077 Acc: 0.0000

Epoch 36/100
----------
train Loss: 2.9513 Acc: 0.0652
val Loss: 2.3284 Acc: 0.0000

Epoch 37/100
----------
train Loss: 2.9721 Acc: 0.0380
val Loss: 2.3975 Acc: 0.0000

Epoch 38/100
----------
train Loss: 2.9826 Acc: 0.0380
val Loss: 2.3214 Acc: 0.0833

Epoch 39/100
----------
train Loss: 2.9735 Acc: 0.0326
val Loss: 2.3876 Acc: 0.0000

Epoch 40/100
----------
train Loss: 2.9612 Acc: 0.0435
val Loss: 2.3194 Acc: 0.0000

Epoch 41/100
----------
train Loss: 2.9479 Acc: 0.0707
val Loss: 2.3397 Acc: 0.0000

Epoch 42/100
----------
train Loss: 2.9855 Acc: 0.0489
val Loss: 2.2628 Acc: 0.0833

Epoch 43/100
----------
train Loss: 2.9617 Acc: 0.0435
val Loss: 2.3105 Acc: 0.0833

Epoch 44/100
----------
train Loss: 2.9678 Acc: 0.0435
val Loss: 2.3475 Acc: 0.0000

Epoch 45/100
----------
train Loss: 2.9978 Acc: 0.0326
val Loss: 2.3121 Acc: 0.0833

Epoch 46/100
----------
train Loss: 2.9631 Acc: 0.0652
val Loss: 2.2540 Acc: 0.1667

Epoch 47/100
----------
train Loss: 2.9613 Acc: 0.0272
val Loss: 2.2788 Acc: 0.0833

Epoch 48/100
----------
train Loss: 2.9713 Acc: 0.0272
val Loss: 2.2627 Acc: 0.0000

Epoch 49/100
----------
train Loss: 2.9681 Acc: 0.0435
val Loss: 2.4135 Acc: 0.0000

Epoch 50/100
----------
train Loss: 2.9493 Acc: 0.0543
val Loss: 2.2882 Acc: 0.0000

Epoch 51/100
----------
train Loss: 2.9806 Acc: 0.0272
val Loss: 2.3411 Acc: 0.0000

Epoch 52/100
----------
train Loss: 2.9612 Acc: 0.0272
val Loss: 2.3398 Acc: 0.0000

Epoch 53/100
----------
train Loss: 2.9794 Acc: 0.0435
val Loss: 2.3660 Acc: 0.0000

Epoch 54/100
----------
train Loss: 2.9578 Acc: 0.0761
val Loss: 2.2724 Acc: 0.0833

Epoch 55/100
----------
train Loss: 2.9556 Acc: 0.0543
val Loss: 2.2508 Acc: 0.0000

Epoch 56/100
----------
train Loss: 2.9648 Acc: 0.0380
val Loss: 2.2830 Acc: 0.0833

Epoch 57/100
----------
train Loss: 2.9801 Acc: 0.0326
val Loss: 2.3244 Acc: 0.0000

Epoch 58/100
----------
train Loss: 2.9574 Acc: 0.0543
val Loss: 2.3250 Acc: 0.0000

Epoch 59/100
----------
train Loss: 2.9822 Acc: 0.0380
val Loss: 2.3394 Acc: 0.0833

Epoch 60/100
----------
train Loss: 2.9928 Acc: 0.0272
val Loss: 2.3060 Acc: 0.0833

Epoch 61/100
----------
train Loss: 2.9746 Acc: 0.0326
val Loss: 2.2304 Acc: 0.0000

Epoch 62/100
----------
train Loss: 2.9595 Acc: 0.0543
val Loss: 2.3114 Acc: 0.0000

Epoch 63/100
----------
train Loss: 2.9732 Acc: 0.0543
val Loss: 2.2681 Acc: 0.0000

Epoch 64/100
----------
train Loss: 2.9688 Acc: 0.0707
val Loss: 2.2953 Acc: 0.0000

Epoch 65/100
----------
train Loss: 2.9638 Acc: 0.0543
val Loss: 2.3149 Acc: 0.0000

Epoch 66/100
----------
train Loss: 2.9749 Acc: 0.0272
val Loss: 2.2717 Acc: 0.0833

Epoch 67/100
----------
train Loss: 2.9705 Acc: 0.0380
val Loss: 2.3136 Acc: 0.0000

Epoch 68/100
----------
train Loss: 2.9669 Acc: 0.0598
val Loss: 2.3525 Acc: 0.0833

Epoch 69/100
----------
train Loss: 2.9736 Acc: 0.0380
val Loss: 2.3059 Acc: 0.0000

Epoch 70/100
----------
train Loss: 2.9509 Acc: 0.0380
val Loss: 2.3224 Acc: 0.0000

Epoch 71/100
----------
train Loss: 2.9670 Acc: 0.0435
val Loss: 2.3885 Acc: 0.0000

Epoch 72/100
----------
train Loss: 2.9883 Acc: 0.0109
val Loss: 2.3839 Acc: 0.0000

Epoch 73/100
----------
train Loss: 2.9600 Acc: 0.0489
val Loss: 2.2624 Acc: 0.0000

Epoch 74/100
----------
train Loss: 2.9622 Acc: 0.0489
val Loss: 2.2768 Acc: 0.0000

Epoch 75/100
----------
train Loss: 2.9699 Acc: 0.0326
val Loss: 2.2774 Acc: 0.0000

Epoch 76/100
----------
train Loss: 2.9763 Acc: 0.0489
val Loss: 2.3497 Acc: 0.0000

Epoch 77/100
----------
train Loss: 2.9438 Acc: 0.0380
val Loss: 2.2421 Acc: 0.0000

Epoch 78/100
----------
train Loss: 2.9784 Acc: 0.0598
val Loss: 2.2838 Acc: 0.0000

Epoch 79/100
----------
train Loss: 2.9705 Acc: 0.0326
val Loss: 2.3115 Acc: 0.0000

Epoch 80/100
----------
train Loss: 2.9812 Acc: 0.0326
val Loss: 2.3237 Acc: 0.0000

Epoch 81/100
----------
train Loss: 2.9534 Acc: 0.0435
val Loss: 2.2675 Acc: 0.0833

Epoch 82/100
----------
train Loss: 2.9821 Acc: 0.0489
val Loss: 2.3357 Acc: 0.0000

Epoch 83/100
----------
train Loss: 2.9728 Acc: 0.0217
val Loss: 2.3066 Acc: 0.0000

Epoch 84/100
----------
train Loss: 2.9879 Acc: 0.0326
val Loss: 2.2450 Acc: 0.0833

Epoch 85/100
----------
train Loss: 2.9540 Acc: 0.0652
val Loss: 2.2688 Acc: 0.0000

Epoch 86/100
----------
train Loss: 2.9717 Acc: 0.0272
val Loss: 2.3908 Acc: 0.0000

Epoch 87/100
----------
train Loss: 2.9610 Acc: 0.0489
val Loss: 2.2722 Acc: 0.0000

Epoch 88/100
----------
train Loss: 2.9408 Acc: 0.0652
val Loss: 2.2503 Acc: 0.0833

Epoch 89/100
----------
train Loss: 2.9878 Acc: 0.0435
val Loss: 2.3507 Acc: 0.0833

Epoch 90/100
----------
train Loss: 2.9419 Acc: 0.0707
val Loss: 2.2950 Acc: 0.0000

Epoch 91/100
----------
train Loss: 2.9584 Acc: 0.0380
val Loss: 2.4055 Acc: 0.0000

Epoch 92/100
----------
train Loss: 2.9534 Acc: 0.0380
val Loss: 2.2548 Acc: 0.0000

Epoch 93/100
----------
train Loss: 2.9785 Acc: 0.0380
val Loss: 2.3115 Acc: 0.0833

Epoch 94/100
----------
train Loss: 2.9588 Acc: 0.0815
val Loss: 2.2968 Acc: 0.0000

Epoch 95/100
----------
train Loss: 2.9861 Acc: 0.0543
val Loss: 2.2684 Acc: 0.0833

Epoch 96/100
----------
train Loss: 2.9548 Acc: 0.0435
val Loss: 2.3082 Acc: 0.0000

Epoch 97/100
----------
train Loss: 2.9834 Acc: 0.0217
val Loss: 2.2860 Acc: 0.0000

Epoch 98/100
----------
train Loss: 2.9610 Acc: 0.0435
val Loss: 2.3605 Acc: 0.0000

Epoch 99/100
----------
train Loss: 2.9666 Acc: 0.0598
val Loss: 2.2922 Acc: 0.0000

Epoch 100/100
----------
train Loss: 2.9891 Acc: 0.0272
val Loss: 2.3154 Acc: 0.0000

Training complete in 20m 49s
Best val Acc: 0.166667